In [ ]:
stim_window = (4000,5200) #es la ventana de tiempo del vdeo que quiero analizar, el original estaba de 100-5500.
res_dir = r"D:\Valen Azar\Escape\Aislados\Analisis_Aislados"

In [ ]:
import os
import pandas as pd

# Replace these with your actual directories
csv_directory = r'D:\Valen Azar\Escape\Aislados\Analisis_Aislados' #la dirección donde están los csv
datos_directory = r'D:\Valen Azar\Escape\Aislados\Analisis_Aislados'

# Initialize an empty list to store dictionaries
all_data = []

# Read Datos Excel file and keep only relevant columns
datos_path = os.path.join(datos_directory, 'Datos_Todos.xlsx')
datos_df = pd.read_excel(datos_path)

# Go through each CSV file in the directory
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        # Extract video name from filename
        video_name = filename.split('DLC')[0] + "DLC_resnet50_Arena Escape ActualizadaApr22shuffle1_300000.csv"
        

        # Find the metadata for this video in the Datos DataFrame
        metadata = datos_df[datos_df['Video'] == video_name].iloc[0].to_dict()

        # Read the CSV file without header information
        csv_path = os.path.join(csv_directory, filename)
        temp_df = pd.read_csv(csv_path, header=None, skiprows=[0])

        # Create new column names based on the second and third rows from the CSV
        # (which are now the first and second rows in temp_df since we skipped the original first row)
        new_columns = [f"{a}_{b}" for a, b in zip(temp_df.iloc[0], temp_df.iloc[1])]

        # Drop the first two rows, which are now part of the column names
        temp_df = temp_df.drop([0, 1]).reset_index(drop=True)

        # Assign the new column names to temp_df
        temp_df.columns = new_columns

        # Drop columns that have 'likelihood' in their name
        temp_df = temp_df[[col for col in temp_df.columns if 'likelihood' not in col]]

        # Create a dictionary to hold DataFrame and metadata
        video_dict = {
            'video_name': video_name,
            'tracking_data': temp_df,
            'metadata': metadata
        }

        # Append the dictionary to the list
        all_data.append(video_dict)

In [ ]:
os.chdir(res_dir)

In [ ]:
# Check if all_data is populated
if len(all_data) == 0:
    print("Error: all_data list is empty.")
else:
    print(f"Successfully processed {len(all_data)} CSV files.")

# Loop through each dictionary in all_data to validate its contents
for i, video_dict in enumerate(all_data):
    print(f"Checking dictionary {i + 1}...")

    # Check video_name
    if not isinstance(video_dict.get('video_name', None), str):
        print(f"  Error: video_name in dictionary {i + 1} is not a string.")

    # Check tracking_data
    tracking_data = video_dict.get('tracking_data', None)
    if tracking_data is None or not isinstance(tracking_data, pd.DataFrame) or tracking_data.empty:
        print(f"  Error: tracking_data in dictionary {i + 1} is not a valid DataFrame.")

    # Check metadata
    metadata = video_dict.get('metadata', None)
    if metadata is None or not all(key in metadata for key in ['Video', 'Visual', 'Auditivo']):
        print(f"  Error: metadata in dictionary {i + 1} is missing one or more keys.")
    else:
        print(f"  Metadata for {video_dict['video_name']}: {metadata}")

print("Check complete.")


In [ ]:
# Initialize an empty dictionary to store trials grouped by visual and auditory intensities
trials = {}

# Iterate over each video_dict in all_data
for video_dict in all_data:
    # Extract visual and auditory intensities from the metadata
    visual_intensity = video_dict['metadata']['Visual']
    auditory_intensity = video_dict['metadata']['Auditivo']
    
    # Create a key based on the combination of visual and auditory intensities
    key = f"{visual_intensity}_{auditory_intensity}"
    
    # Add the video_dict to the corresponding entry in the trials dictionary
    if key not in trials:
        trials[key] = []
    trials[key].append(video_dict)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import sample
import math

def vector_magnitude(v):
    return math.sqrt(v[0]**2 + v[1]**2)

def dot_product(v1, v2):
    return v1[0]*v2[0] + v1[1]*v2[1]

def angle_between_vectors(v1, v2):
    # Handle zero vectors
    if vector_magnitude(v1) == 0 or vector_magnitude(v2) == 0:
        return None  # Angle is undefined for zero vectors

    # Calculate cosine of the angle
    cos_theta = dot_product(v1, v2) / (vector_magnitude(v1) * vector_magnitude(v2))
    
    # Clip the value to the range [-1, 1] to handle any possible rounding errors
    cos_theta = min(1.0, max(-1.0, cos_theta))
    
    # Calculate the angle in radians
    angle = math.acos(cos_theta)
    
    # Convert the angle to degrees for easier interpretation
    angle_degrees = math.degrees(angle)
    
    return angle_degrees

def calculate_features(df):
    # Convert columns to numeric type if they are not already
    numeric_columns = [
        'ojo izquierdo_x', 'ojo izquierdo_y', 'ojo derecho_x', 'ojo derecho_y', 
        'vejiga anterior_x', 'vejiga anterior_y', 'vejiga posterior_x', 'vejiga posterior_y', 
        'cola 1_x', 'cola 1_y', 'cola 2_x', 'cola 2_y', 
        'cola 3_x', 'cola 3_y', 'cola 4_x', 'cola 4_y', 
        'cola 5_x', 'cola 5_y', 'cola 6_x', 'cola 6_y'
    ]
    
    for column in numeric_columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    
    # Calculate the mean point between both eyes
    df['eye_mean_x'] = (df['ojo izquierdo_x'] + df['ojo derecho_x']) / 2
    df['eye_mean_y'] = (df['ojo izquierdo_y'] + df['ojo derecho_y']) / 2
    
    # Calculate the mean point between the last two tail points
    df['tail_mean_x'] = (df['cola 5_x'] + df['cola 6_x']) / 2
    df['tail_mean_y'] = (df['cola 5_y'] + df['cola 6_y']) / 2
    
    # Calculate Instant head and tail velocity
    df['head_velocity'] = np.sqrt((df['eye_mean_x'].diff())**2 + (df['eye_mean_y'].diff())**2)
    df['tail_velocity'] = np.sqrt((df['tail_mean_x'].diff())**2 + (df['tail_mean_y'].diff())**2)
    
    # Calculate Angular Velocity
    current_head_vector = [df["vejiga posterior_x"] - df["eye_mean_x"], df["vejiga posterior_y"] - df["eye_mean_y"]]
    current_head_vector = np.asarray(current_head_vector).astype("float")
    current_head_vector = current_head_vector.T
    
    # Calculate the magnitudes of these vectors
    magnitudes = np.linalg.norm(current_head_vector, axis=1)

    # Calculate the dot product between each pair of subsequent vectors
    dot_products = np.sum(current_head_vector[:-1] * current_head_vector[1:], axis=1)

    # Calculate the magnitudes of pairs of subsequent vectors
    magnitude_pairs = magnitudes[:-1] * magnitudes[1:]

    # Calculate the angles in radians
    angles = np.arccos(dot_products / magnitude_pairs)
    angles = np.concatenate([angles, [angles[-1]]])
    
    # Calculate the angle between the current vector and the previous vector
    df['angular_velocity'] = angles
    
    # Calculate Sum of Tail Angles
    tail_angles = []
    for i in range(1, 5):
        vec1 = df[f'cola {i}_x'] - df[f'cola {i+1}_x'], df[f'cola {i}_y'] - df[f'cola {i+1}_y']
        vec2 = df[f'cola {i+1}_x'] - df[f'cola {i+2}_x'], df[f'cola {i+1}_y'] - df[f'cola {i+2}_y']
        angle = np.arctan2(vec2[1], vec2[0]) - np.arctan2(vec1[1], vec1[0])
        tail_angles.append(angle)
    df['sum_tail_angles'] = np.sum(tail_angles, axis=0)
    
    # Calculate Instant velocity for tail6 point
    df['tail6_velocity'] = np.sqrt((df['cola 3_x'].diff())**2 + (df['cola 3_y'].diff())**2)
    
    df['head_tail_distance'] = np.sqrt((df['eye_mean_x'] - df['tail_mean_x'])**2 + (df['eye_mean_y'] - df['tail_mean_y'])**2)
    
    df['head_acceleration'] = df['head_velocity'].diff().fillna(0)
    df['tail_acceleration'] = df['tail_velocity'].diff().fillna(0)

    
    df['angular_acceleration'] = df['angular_velocity'].diff().fillna(0)

    
    df['head_jerk'] = df['head_acceleration'].diff().fillna(0)
    df['tail_jerk'] = df['tail_acceleration'].diff().fillna(0)

    
    curvatures = []
    for i in range(1, 5):
        x1, y1 = df[f'cola {i}_x'], df[f'cola {i}_y']
        x2, y2 = df[f'cola {i+1}_x'], df[f'cola {i+1}_y']
        x3, y3 = df[f'cola {i+2}_x'], df[f'cola {i+2}_y']

        numerator = (x1 - x2) * (y2 - y3) - (y1 - y2) * (x2 - x3)
        denominator = np.sqrt(((x1 - x2)**2 + (y1 - y2)**2) * ((x2 - x3)**2 + (y2 - y3)**2) * ((x1 - x3)**2 + (y1 - y3)**2))

        curvature = numerator / denominator
        curvatures.append(curvature)

    df['sum_curvature'] = np.sum(curvatures, axis=0)

    # Compute the velocity vector components
    delta_x = df['eye_mean_x'].diff()
    delta_y = df['eye_mean_y'].diff()

    # Compute the heading vector components
    heading_x = df['vejiga posterior_x'] - df['eye_mean_x']
    heading_y = df['vejiga posterior_y'] - df['eye_mean_y']

    # Compute the angle
    df['angle_velocity_heading'] = np.arccos((delta_x * heading_x + delta_y * heading_y) / 
                                              (np.sqrt(delta_x**2 + delta_y**2) * np.sqrt(heading_x**2 + heading_y**2)))
    
    df['angle_velocity_heading'] = df['angle_velocity_heading'].fillna(0)
    
    df['curvature_rate'] = df['sum_curvature'].diff().fillna(0)

    return df[['head_velocity', 'tail_velocity', 'angular_velocity',
               'sum_tail_angles', 'tail6_velocity',
               'head_tail_distance', 'head_acceleration', 'tail_acceleration',
               'angular_acceleration', 'head_jerk', 'tail_jerk', 'sum_curvature',
               'angle_velocity_heading', 'curvature_rate']]


In [ ]:
!pip install scipy

In [ ]:
from scipy.stats import norm
threshold_probability = 0.000000000000000001  # You can change this threshold later

# Calculate features and find events for each trial
for stimulus, trial_list in trials.items():
    for trial in trial_list:
        tracking_data = trial['tracking_data']
        new_features_df = calculate_features(tracking_data)
        
        # Step 1: Calculate the median velocity
        median_velocity = new_features_df['head_velocity'].median()
        
        # Step 2: Create a mirrored distribution around the median
        lower_half = new_features_df['head_velocity'][new_features_df['head_velocity'] < median_velocity]
        mirrored_distribution = np.concatenate([lower_half, 2 * median_velocity - lower_half])
        
        # Step 3: Calculate mean and standard deviation
        mu, std = np.mean(mirrored_distribution), np.std(mirrored_distribution)
        
        # Step 4: Calculate probability for each timepoint
        cdf_values = norm.cdf(new_features_df['head_velocity'], mu, std)
        probabilities = 1 - cdf_values  # Tail probabilities
        
        # Step 5: Identify events
        # Initialize variables
        in_event = False
        event_times = []
        event_ends = []
        current_event_start = None
        above_threshold_counter = 0  # Initialize counter
        n = 1  # ERA 3 #Number of frames needed to confirm end of event #era 20 #Esto quiere decir que si yo tengo al pez que se mueve con distinta velocidad, cuando la misma traspasa cierto umbral, para que sea un evento tiene que estar como minimo 5 frames. Cuando vuelve a traspasar el umbral, es un nuevo evento.
        velocity_threshold = 1  # Replace with your specific value #era 1 #Umbral de velocidad para considerar un evento.
        event_valid = False  # Initialize event validity flag

        for i, prob in enumerate(probabilities):
            velocity = new_features_df['head_velocity'].iloc[i]

            if prob < threshold_probability:
                above_threshold_counter = 0  # Reset counter
                if not in_event:
                    in_event = True
                    event_valid = False  # Reset event validity flag
                    current_event_start = i  # Mark the start of the new event

            if in_event:
                if velocity > velocity_threshold:
                    event_valid = True  # Mark event as valid

            if prob >= threshold_probability:
                if in_event:
                    above_threshold_counter += 1  # Increment counter
                    if above_threshold_counter >= n:
                        in_event = False
                        above_threshold_counter = 0  # Reset counter

                        if event_valid:  # Only add event if it's valid
                            event_ends.append(i)  # Mark the end of the event
                            event_times.append(current_event_start)  # Save the start time of the event

        # Save features and event times
        trial['features'] = new_features_df
        trial['event_times'] = event_times # esto es el frame de inicio de un evento
        trial['event_ends'] = event_ends # esto es el frame final de un evento. Cuando hago event_ends - event_times, me da la duración del evento


In [ ]:
# Visualización del resultado
back_window = 5
forw_window = 25
x_limit = 5500  # Límite del eje X
y_limit = 12  # Definimos el límite en el eje Y

for stimulus, trial_list in trials.items():
    random_trials = sample(trial_list, min(5, len(trial_list)))
    
    for i, trial in enumerate(random_trials):
        plt.figure(figsize=(12, 6))
        features_df = trial['features']
        event_times = trial['event_times']
        
        for event_time in event_times:
            if event_time <= x_limit:  # Solo mostrar eventos dentro del rango
                maxvel = features_df['head_velocity'][event_time - back_window: event_time + forw_window].max()
                if maxvel > 4:
                    plt.axvline(x=event_time, color='red', linestyle='--', linewidth=2, label='Event' if event_time == event_times[0] else "")
                    plt.text(event_time, maxvel, str(event_time), color='red', fontsize=12, ha='center')

        plt.plot(features_df['head_velocity'], label='Instant Head Velocity')
        plt.title(f'Stimulus: {stimulus}, Trial: {trial["video_name"]}')
        plt.xlabel('Timepoint')
        plt.ylabel('Normalized Instant Head Velocity')
        plt.legend()
        plt.xlim(0, x_limit)  # Establecer el límite del eje X
        plt.ylim(0, y_limit)  # Ajustar el límite del eje Y a 12
        plt.savefig(f"Event_example_{trial['video_name']}.pdf", format='pdf', dpi=300, bbox_inches='tight')
        plt.show()

In [ ]:
trial = trials["160_0.0"][0] #acá voy variando entre las combinaciones de estímulos (160-0);(230-0) etc.
features = trial['features'] #las features son las variables que el codigo calcula con el csv y que están detalladas más arriba.
feature_list = []
for feature_name, feature_values in features.items():
    feature_list.append(feature_name)
    print(feature_name) #me da los nombres de las variables

In [ ]:
import numpy as np
import pandas as pd

#back_window = 20
#forw_window = 40

back_window = 5
forw_window = 25

for stimulus, trial_list in trials.items():
    for trial in trial_list:
        features = trial['features']
        event_times = trial['event_times']
        
        event_features_over_time = []
        valid_event_times = []
        
        for event_time in event_times:
            
            # Exclude events too close to the trial limits
            if event_time < back_window or event_time > len(features['head_velocity']) - forw_window:
                continue
            
            maxvel = features['head_velocity'][event_time - back_window: event_time + forw_window].max()
            
            if maxvel < 3 or maxvel > 25:   #velocidad original entre < 3 y > 15.
                continue
            
            valid_event_times.append(event_time)
            time_window_features = []
            
            for feature_name, feature_values in features.items():
                # Extract 20 frames before to 40 frames after the event
                time_window = feature_values[event_time - back_window: event_time + forw_window]
                time_window_features.append(time_window)
            
            event_features_over_time.append(time_window_features)
        
        trial['event_features_over_time'] = np.array(event_features_over_time)
        trial['valid_event_times'] = np.array(valid_event_times)

In [ ]:
# import matplotlib.pyplot as plt

# # Initialize an empty list to store max head velocities
# max_head_velocities = []

# # Loop through each trial based on the stimulus types
# for stimulus_type, trial_list in trials.items():
#     for trial in trial_list:
#         # Access the 'event_features_over_time' array for the trial
#         event_features_over_time = trial['event_features_over_time']
        
#         head_velocity_idx = feature_list.index('angular_acceleration')
        
#         # Loop through each event and find the maximum head velocity
#         for event in range(event_features_over_time.shape[0]):
#             max_head_velocity = event_features_over_time[event, head_velocity_idx, :].max()
#             max_head_velocities.append(max_head_velocity)

# # Create the histogram
# plt.hist(max_head_velocities, bins=50)
# plt.title('Histogram of Max Head Velocities')
# plt.xlabel('Max Head Velocity')
# plt.ylabel('Frequency')
# #plt.ylim(0,400)
# plt.show()

In [ ]:
selected_features = ['head_velocity', 'angular_velocity', 'head_tail_distance', 'sum_curvature']

for stimulus, trial_list in trials.items():
    for trial in trial_list:
        event_features = []
        
        for event_time_window in trial['event_features_over_time']:
            event_feature_params = []
            max_frame_diffs = []  # List to store the differences in frame numbers for the selected features
            
            feature_to_max_frame = {}  # Dictionary to store the frame of max value for each feature
            
            for i, feature_name in enumerate(trial['features'].keys()):
                time_window = event_time_window[i]
                
                # Your existing code to calculate event_feature_params
                if feature_name == 'head_tail_distance':
                    event_feature_params.extend([np.min(time_window), np.mean(time_window)])
                elif feature_name in ['sum_tail_angles', 'head_acceleration', 'tail_acceleration', 
                                      'angular_acceleration', 'head_jerk', 'tail_jerk', 
                                      'sum_curvature', 'curvature_rate']:
                    event_feature_params.extend([np.max(time_window) - np.min(time_window), 
                                                 np.max(np.abs(time_window)), 
                                                 np.mean(np.abs(time_window))])
                else:
                    event_feature_params.extend([np.max(time_window), np.mean(time_window)])
                
                # Check if the feature is one of the selected features
                if feature_name in selected_features:
                    max_frame = np.argmax(time_window)  # Find the frame where the max value occurs
                    feature_to_max_frame[feature_name] = max_frame  # Store it in the dictionary
            
            # Calculate the differences in max frame numbers for each pair of selected features
            for i, feature1 in enumerate(selected_features):
                for j, feature2 in enumerate(selected_features[i+1:]):
                    max_frame_diff = abs(feature_to_max_frame[feature1] - feature_to_max_frame[feature2])
                    max_frame_diffs.append(max_frame_diff)
            
            # Append both the existing and new parameters for this event
            event_feature_params.extend(max_frame_diffs)
            event_features.append(event_feature_params)
        
        # Update the event_features array for the current trial
        trial['event_features'] = np.array(event_features)


In [ ]:
# for stimulus, trial_list in trials.items():
#     for trial in trial_list:
#         event_features = []
        
#         for event_time_window in trial['event_features_over_time']:
#             event_feature_params = []
            
#             for i, feature_name in enumerate(trial['features'].keys()):
#                 time_window = event_time_window[i]
                
#                 if feature_name == 'head_tail_distance':
#                     event_feature_params.extend([np.min(time_window), np.mean(time_window)])
#                 elif feature_name in ['sum_tail_angles', 'head_acceleration', 'tail_acceleration', 
#                                       'angular_acceleration', 'head_jerk', 'tail_jerk', 
#                                       'sum_curvature', 'curvature_rate']:
#                     event_feature_params.extend([np.max(time_window) - np.min(time_window), 
#                                                  np.max(np.abs(time_window)), 
#                                                  np.mean(np.abs(time_window))])
#                 else:
#                     event_feature_params.extend([np.max(time_window), np.mean(time_window)])
            
#             event_features.append(event_feature_params)
        
#         trial['event_features'] = np.array(event_features)


In [ ]:
# import matplotlib.pyplot as plt

# # Initialize an empty dictionary to collect frame differences for each pair of features
# frame_diffs_dict = {}

# for stimulus, trial_list in trials.items():
#     for trial in trial_list:
#         # Assuming the last N columns in event_features are the frame differences,
#         # where N is the number of combinations of selected features
#         num_combinations = len(selected_features) * (len(selected_features) - 1) // 2
#         if trial['event_features'].shape[0] > 0:
#             frame_diffs = trial['event_features'][:, -num_combinations:]

#             # If this is the first trial, initialize the dictionary
#             if not frame_diffs_dict:
#                 idx = 0
#                 for i, feature1 in enumerate(selected_features):
#                     for feature2 in selected_features[i+1:]:
#                         frame_diffs_dict[f"{feature1}-{feature2}"] = []
#                         idx += 1

#             # Append the frame differences to the respective lists
#             for idx, feature_pair in enumerate(frame_diffs_dict.keys()):
#                 feature1, feature2 = feature_pair.split('-')
#                 frame_diffs_dict[f"{feature1}-{feature2}"].extend(frame_diffs[:, idx])

# # Plotting the histograms
# for feature_pair, diff_values in frame_diffs_dict.items():
#     plt.figure()
#     plt.hist(diff_values, bins=15)
#     plt.title(f"Frame Differences for {feature_pair}")
#     plt.xlabel("Frame Difference")
#     plt.ylabel("Frequency")
#     plt.show()
   

In [ ]:
all_events = []

for stimulus, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        all_events.extend(trial['event_features'])

all_events_array = np.array(all_events)

# Normalize
mean_vals = np.mean(all_events_array, axis=0)
std_vals = np.std(all_events_array, axis=0)
normalized_events = (all_events_array - mean_vals) / std_vals


In [ ]:
normalized_events.shape #me da el número de eventos (519) y el número de dimensiones (42).

In [ ]:
# Opciones: TSNE, PCA y KernelPCA. La idea es ir jugando con los parametros de tal forma que se observen clusters. 
# Los puntos rojos son los eventos que yo identifiqué, si en los gráficos se me juntan los puntos rojos es probable que 
# haya un cluster ahí.

#from sklearn.manifold import TSNE

#tsne = TSNE(n_components=2, perplexity = 25)
#low_dimensional_data = tsne.fit_transform(normalized_events)

#from umap import UMAP

#umap = UMAP(n_components=2, n_neighbors=20, min_dist=0.3)
#low_dimensional_data = umap.fit_transform(normalized_events)

#from sklearn.decomposition import PCA

#pca = PCA(n_components=2) #el numero de componentes original es de 2.
#low_dimensional_data = pca.fit_transform(normalized_events)

from sklearn.decomposition import KernelPCA #EL QUE ESTUVE USANDO Y PUSE EN LA TESIS

kernel_pca = KernelPCA(n_components=2, kernel="rbf", gamma = 0.015) #Original gamma=0.015 # Tengo que jugar con el gamma para ver si se forman clusters o no. O se alejan o se juntan.
low_dimensional_data = kernel_pca.fit_transform(normalized_events)


In [ ]:
index_counter = 0

for stimulus, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        num_events = len(trial['event_features'])
        trial['low_D_representation'] = low_dimensional_data[index_counter:index_counter + num_events]
        index_counter += num_events

In [ ]:
for stimulus, trial_list in trials.items():
    for trial in trial_list:
        F_escape = trial['metadata']['F_escape']
        valid_event_times = trial['valid_event_times']

        is_escape = np.zeros(len(valid_event_times), dtype=bool)

        if not np.isnan(F_escape) and len(valid_event_times) > 0:
            closest_event = np.argmin(np.abs(valid_event_times - F_escape))
            if np.abs(valid_event_times[closest_event] - F_escape) < 50:
                is_escape[closest_event] = True

        trial['is_escape'] = is_escape

        trial['condicion'] = trial['metadata']['condicion'] #social o aislado

In [ ]:
trial.keys()

In [ ]:
#plt.savefig(f"01 - Diff in N of Active Neurons and Diff in Mean Neural Act Over Time ({active_stim_code} - {ctrl_stim_code}).pdf", format='pdf', dpi=300, bbox_inches='tight')

In [ ]:
## CARGA DE DATOS CON FILTRO # NO USAR POR EL MOMENTO, NO VAMOS A VER EN CADA ESTÍMULO

#import matplotlib.pyplot as plt
#import numpy as np

## Initialize empty lists to collect data points and colors
#concatenated_low_D = []
#concatenated_low_D_in_range = []
#concatenated_colors = []
#concatenated_colors_in_range = []

## Iterate over trials and collect data points and colors
#for stimulus, trial_list in trials.items():
    
    # Filtrar solo los estímulos "160-0.0"  
    #if stimulus != "160-0.0":
     #    continue
    # Si cambio el 'and' por 'or' me va a dar todos los videos que contengan o 230 o 0. (original)
    #if not ("255" in stimulus and "0.04" in stimulus):
     #   continue
        
    #for trial in trial_list:
     #   if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
      #      continue
        # Si quiero puntos de los aislados solo pongo: if trial['condicion'] != 'aislado'. Si quiero aislados y sociales no corro esta parte (la pongo con asteriscos)
        #if trial['condicion'] != 'social':
         #   continue
        
       # within_frame_range = (trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1])

        ## Data for first subplot (only within_frame_range)
        #concatenated_low_D_in_range.extend(trial['low_D_representation'][within_frame_range])
        #concatenated_colors_in_range.extend(['red' if escape and trial['condicion'] == 'social' else 'green' if escape and trial['condicion'] == 'aislado' else 'blue' for escape in trial['is_escape'][within_frame_range]])

        ## Data for second subplot (all data)
        #concatenated_low_D.extend(trial['low_D_representation'])
        #concatenated_colors.extend(['red' if escape and trial['condicion'] == 'social' else 'green' if escape and trial['condicion'] == 'aislado' else 'blue' for escape in trial['is_escape']])
        ## 'red' if escape and trial['condicion'] == 'social' else 'green' if escape and trial['condicion'] == 'aislado' else 'blue'. Sociales son rojos, aislados verdes, lo otro es azul que son cstart no anotados.

## Convert lists to NumPy arrays for consistency
#concatenated_low_D_in_range = np.array(concatenated_low_D_in_range)
#concatenated_colors_in_range = np.array(concatenated_colors_in_range)
#concatenated_low_D = np.array(concatenated_low_D)
#concatenated_colors = np.array(concatenated_colors)

## Check if lengths match (for debugging, can remove later)
#assert len(concatenated_low_D) == len(concatenated_colors), "Lengths must match!"

## Create subplots
#fig, axs = plt.subplots(1, 2, figsize=(24, 12), sharey=True)

## First subplot (only within_frame_range)
#axs[0].scatter(concatenated_low_D_in_range[:, 0][concatenated_colors_in_range == "blue"], concatenated_low_D_in_range[:, 1][concatenated_colors_in_range == "blue"], color="blue", alpha=0.3, s=10)
#axs[0].scatter(concatenated_low_D_in_range[:, 0][concatenated_colors_in_range == "red"], concatenated_low_D_in_range[:, 1][concatenated_colors_in_range == "red"], color="red", alpha=0.6, s=40)
#axs[0].scatter(concatenated_low_D_in_range[:, 0][concatenated_colors_in_range == "green"], concatenated_low_D_in_range[:, 1][concatenated_colors_in_range == "green"], color="green", alpha=0.6, s=40)
#axs[0].set_title("Within Stim Range", fontsize=16)
#axs[0].tick_params(axis='both', which='major', labelsize=14)

## Second subplot (all data)
#axs[1].scatter(concatenated_low_D[:, 0][concatenated_colors == "blue"], concatenated_low_D[:, 1][concatenated_colors == "blue"], color="blue", alpha=0.3, s=10)
#axs[1].scatter(concatenated_low_D[:, 0][concatenated_colors == "red"], concatenated_low_D[:, 1][concatenated_colors == "red"], color="red", alpha=0.6, s=40)
#axs[1].scatter(concatenated_low_D[:, 0][concatenated_colors == "green"], concatenated_low_D[:, 1][concatenated_colors == "green"], color="green", alpha=0.6, s=40)
#axs[1].set_title("All Events", fontsize=16)
#axs[1].tick_params(axis='both', which='major', labelsize=14)

##plt.savefig(f"01 - 2D representation of escape events.pdf", format='pdf', dpi=300, bbox_inches='tight')
#plt.show()


In [ ]:
# CARGA DE DATOS SIN FILTRO - Todos los animales aislados y sociales

import matplotlib.pyplot as plt
import numpy as np

# Initialize empty lists to collect data points and colors
concatenated_low_D = []
concatenated_low_D_in_range = []
concatenated_colors = []
concatenated_colors_in_range = []

# Iterate over trials and collect data points and colors. la parte del tipo de estimulo es nueva, lo de 'if not () continue'.
for stimulus, trial_list in trials.items():
    #if not ("230" in stimulus and "0" in stimulus):
     #   continue
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
            # Si quiero puntos de los aislados solo pongo: if trial['condicion'] != 'aislado'. Si quiero aislados y sociales no corro esta parte (la pongo con numeral)
        if trial['condicion'] != 'aislado':
            continue
        within_frame_range = (trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1])

        # Data for first subplot (only within_frame_range)
        concatenated_low_D_in_range.extend(trial['low_D_representation'][within_frame_range])
        concatenated_colors_in_range.extend(['red' if escape and trial['condicion'] == 'social' else 'green' if escape and trial['condicion'] == 'aislado' else 'blue' for escape in trial['is_escape'][within_frame_range]])

        # Data for second subplot (all data)
        concatenated_low_D.extend(trial['low_D_representation'])
        concatenated_colors.extend(['red' if escape and trial['condicion'] == 'social' else 'green' if escape and trial['condicion'] == 'aislado' else 'blue' for escape in trial['is_escape']])
        # 'red' if escape and trial['condicion'] == 'social' else 'green' if escape and trial['condicion'] == 'aislado' else 'blue'. Sociales son rojos, aislados verdes, lo otro es azul que son cstart no anotados.

# Convert lists to NumPy arrays for consistency
concatenated_low_D_in_range = np.array(concatenated_low_D_in_range)
concatenated_colors_in_range = np.array(concatenated_colors_in_range)
concatenated_low_D = np.array(concatenated_low_D)
concatenated_colors = np.array(concatenated_colors)

# Check if lengths match (for debugging, can remove later)
assert len(concatenated_low_D) == len(concatenated_colors), "Lengths must match!"

# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(24, 12), sharey=True)

# First subplot (only within_frame_range) - PARA SACAR LOS PUNTOS AZULES SOLO PONGO # ANTES DE AXS [0] PARA LOS BLUE
#axs[0].scatter(concatenated_low_D_in_range[:, 0][concatenated_colors_in_range == "blue"], concatenated_low_D_in_range[:, 1][concatenated_colors_in_range == "blue"], color="blue", alpha=0.3, s=10)
axs[0].scatter(concatenated_low_D_in_range[:, 0][concatenated_colors_in_range == "red"], concatenated_low_D_in_range[:, 1][concatenated_colors_in_range == "red"], color="red", alpha=0.6, s=40)
axs[0].scatter(concatenated_low_D_in_range[:, 0][concatenated_colors_in_range == "green"], concatenated_low_D_in_range[:, 1][concatenated_colors_in_range == "green"], color="green", alpha=0.6, s=40)
axs[0].set_title("Within Stim Range", fontsize=16)
axs[0].tick_params(axis='both', which='major', labelsize=14)

# Second subplot (all data) - PARA SACAR LOS PUNTOS AZULES SOLO PONGO # ANTES DE AXS [0] PARA LOS BLUE
#axs[1].scatter(concatenated_low_D[:, 0][concatenated_colors == "blue"], concatenated_low_D[:, 1][concatenated_colors == "blue"], color="blue", alpha=0.3, s=10)
axs[1].scatter(concatenated_low_D[:, 0][concatenated_colors == "red"], concatenated_low_D[:, 1][concatenated_colors == "red"], color="red", alpha=0.6, s=40)
axs[1].scatter(concatenated_low_D[:, 0][concatenated_colors == "green"], concatenated_low_D[:, 1][concatenated_colors == "green"], color="green", alpha=0.6, s=40)
axs[1].set_title("All Events", fontsize=16)
axs[1].tick_params(axis='both', which='major', labelsize=14)

#plt.savefig(f"01 - 2D representation of escape events.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

# Contar puntos rojos y verdes en el primer subplot (within_frame_range)
red_points_in_range = np.sum(concatenated_colors_in_range == "red")
green_points_in_range = np.sum(concatenated_colors_in_range == "green")

# Contar puntos rojos y verdes en el segundo subplot (all data)
red_points_all = np.sum(concatenated_colors == "red")
green_points_all = np.sum(concatenated_colors == "green")

# Imprimir los resultados
print(f"First subplot (Within Stim Range):")
print(f"Number of red points: {red_points_in_range}")
print(f"Number of green points: {green_points_in_range}\n")

print(f"Second subplot (All Events):")
print(f"Number of red points: {red_points_all}")
print(f"Number of green points: {green_points_all}")



In [ ]:
concatenated_low_D.shape[0]

In [ ]:
# CON ESTE CODIGO ME DA UN GRAFICO EN EL CUAL SELECCIONO LOS PUNTOS DE ARRIBA (NO ME PONE TODOS LOS EVENTOS - ES DECIR NO PONE LOS PUNTOS AZULES).

%matplotlib notebook 

import matplotlib.pyplot as plt
from matplotlib.path import Path
import numpy as np

# Filtrar los puntos que tienen colores rojos y verdes
filtered_points = concatenated_low_D[(concatenated_colors == "red") | (concatenated_colors == "green")]

# Inicializar la figura
fig, ax = plt.subplots(figsize=(10, 10))

# Mostrar solo los puntos filtrados (rojos y verdes)
scatter_plot = ax.scatter(filtered_points[:, 0], filtered_points[:, 1], c='gray', s=5)

coords = []
polygons = []
cluster_count = 0
labels = np.full(filtered_points.shape[0], -1)  # Initialize labels to -1 (unassigned)

def onclick(event):
    global ix, iy, coords
    ix, iy = event.xdata, event.ydata
    print(f'x = {ix}, y = {iy}')

    coords.append((ix, iy))

    polygon = plt.Polygon(coords, fill=None, edgecolor='r')
    ax.add_patch(polygon)

def onkey(event):
    global coords, polygons, cluster_count, labels
    if event.key == 'enter':
        print("Polygon complete. Checking points...")
        
        path = Path(coords)
        mask = path.contains_points(filtered_points)
        
        # Update labels
        labels[mask] = cluster_count
        
        # Save the shape of the polygon
        polygons.append(path)
        
        # Update the color map based on the cluster labels
        scatter_plot.set_array(labels)
        
        # Increase the cluster count
        cluster_count += 1
        
        coords = []  # Reset coords

        plt.draw()

# Conectar eventos para clicks y teclas
cid_click = fig.canvas.mpl_connect('button_press_event', onclick)
cid_key = fig.canvas.mpl_connect('key_press_event', onkey)

plt.show()


In [ ]:
# ORIGINAL - POR EL MOMENTO NO USARLO
# con este codigo entero me da el plot con todos los eventos para seleccionar puntos que podrían ser cluster para mí.

%matplotlib notebook 

import matplotlib.pyplot as plt
from matplotlib.path import Path
import numpy as np

# Sample 2D data (replace this with your t-SNE or other low-dimensional data)
points = concatenated_low_D

fig, ax = plt.subplots(figsize=(10,10))
scatter_plot = ax.scatter(points[:, 0], points[:, 1], c='gray', s=5)

coords = []
polygons = []
cluster_count = 0
labels = np.full(points.shape[0], -1)  # Initialize labels to -1 (unassigned)

def onclick(event):
    global ix, iy, coords
    ix, iy = event.xdata, event.ydata
    print(f'x = {ix}, y = {iy}')

    coords.append((ix, iy))

    polygon = plt.Polygon(coords, fill=None, edgecolor='r')
    ax.add_patch(polygon)

def onkey(event):
    global coords, polygons, cluster_count, labels
    if event.key == 'enter':
        print("Polygon complete. Checking points...")
        
        path = Path(coords)
        mask = path.contains_points(points)
        
        # Update labels
        labels[mask] = cluster_count
        
        # Save the shape of the polygon
        polygons.append(path)
        
        # Update the color map based on the cluster labels
        scatter_plot.set_array(labels)
        
        # Increase the cluster count
        cluster_count += 1
        
        coords = []  # Reset coords

        plt.draw()

cid_click = fig.canvas.mpl_connect('button_press_event', onclick)
cid_key = fig.canvas.mpl_connect('key_press_event', onkey)

plt.show()

In [ ]:
# ORIGINAL
# en este caso "-1" son todos los puntos que NO seleccioné,y "0" son los puntos que SI seleccioné.
%matplotlib inline
np.unique(labels)
# np.sum(labels==0)   #si corro esta línea me debería dar la cantidad de puntos que seleccioné.

In [ ]:
# Celda 21: Asignación de event_selection con verificación
index_counter = 0

for stimulus, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        num_events = len(trial['valid_event_times'])
        
        # Asegúrate de que no excedas el tamaño de labels
        if index_counter + num_events <= len(labels):
            trial['event_selection'] = labels[index_counter:index_counter + num_events]
        else:
            print(f"Index error for stimulus {stimulus}: index_counter={index_counter}, num_events={num_events}, labels_length={len(labels)}")
        
        index_counter += num_events

In [ ]:
# ORIGINAL
# Assuming 'labels' contains the cluster information for all events #ORIGINAL

index_counter = 0

for stimulus, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        #within_frame_range = (trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1])
        num_events = len(trial['valid_event_times'])
        trial['event_selection'] = labels[index_counter:index_counter + num_events]
        index_counter += num_events
        
print(index_counter)

In [ ]:
# Celda 22: Cálculo de max_velocities
for stimulus, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        trial['valid_event_ends'] = []
        
        for valid_event_time in trial['valid_event_times']:
            index = trial['event_times'].index(valid_event_time)
            trial['valid_event_ends'].append(trial['event_ends'][index])
        
        trial['valid_event_ends'] = np.array(trial['valid_event_ends'])
        
        trial['max_velocities'] = []
        
        for i in range(len(trial['valid_event_times'])):
            if i < len(trial['event_selection']):  # Verifica que estés dentro de los límites
                event_time = trial['valid_event_times'][i]
                event_end = trial['valid_event_ends'][i]

                velocities = trial['features']['sum_curvature'][event_time:event_end] # ACÁ SI QUIERO VER OTRAS VARIABLES SOLO CAMBIO LO QUE ESTÁ DESPUES DE "features", es decir donde dice head_velocity (velocities = trial['features']['head_velocity'])
                trial['max_velocities'].append(velocities.max())
        
        trial['max_velocities'] = np.array(trial['max_velocities'])

In [ ]:
# ORIGINAL
# for stimulus, trial_list in trials.items():
#     for trial in trial_list:
#         if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
#             continue
        
#         trial['valid_event_ends'] = []
        
#         for valid_event_time in trial['valid_event_times']:
#             index = trial['event_times'].index(valid_event_time)
#             trial['valid_event_ends'].append(trial['event_ends'][index])
        
#         trial['valid_event_ends'] = np.array(trial['valid_event_ends'])
        
#         trial['max_velocities'] = []
        
#         for i in range(len(trial['valid_event_times'])):
#             event_time = trial['valid_event_times'][i]
#             event_end = trial['valid_event_ends'][i]
            
#             velocities = trial['features']['angular_velocity'][event_time:event_end] #si quiero seleccionar otra variable en lugar de 'head_velocity' pongo otra variable como la velocidad angular etc. Lo que si tengo que cambiar tambien el lugar donde se guarda todo, en lugar de 'max_velocities' debería poner algo asi como 'max_velocidad_angular'.
#             trial['max_velocities'].append(velocities.max())
            
        
#         trial['max_velocities'] = np.array(trial['max_velocities'])

In [ ]:
print(f"Number of max_velocities: {len(trial['max_velocities'])}")
print(f"Number of event_selection: {len(trial['event_selection'])}")
print(f"Total labels: {len(labels)}")

In [ ]:
len(trial['event_selection']== selected)

In [ ]:
len(trial['max_velocities'])

In [ ]:
# Celda 23: Obtener la distribución de max_velocities
import matplotlib.pyplot as plt

selected = 1  # Elige qué grupo de puntos seleccionado quieres plotear.
all_max_velocities = []

for stimulus, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
            
        # Asegúrate de que estés accediendo a los índices correctos
        all_max_velocities.extend(trial['max_velocities'][trial['event_selection'] == selected])  # Filtrado
        
all_max_velocities = np.array(all_max_velocities)

plt.figure()
plt.hist(all_max_velocities[all_max_velocities < 20], bins=40)
plt.xlabel('suma de la curvatura')  # Cambia según la variable que estés usando
plt.show()

In [ ]:
#ORIGINAL
import matplotlib.pyplot as plt #con este codigo voy a obtener un histograma con la distribución de las velocidades o de la variable que quiero.

selected = 0 #acá voy a elegir que grupo de puntos seleccionado quiero plotear. '0' es el primer grupo de puntos seleccionado, si quiero plotear el segundo pongo '1', el tercero es '2'.
all_max_velocities = []

for stimulus, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
            
        
        all_max_velocities.extend(trial['max_velocities'][trial['event_selection'] == selected]) #supuestamente acá está el error cuando quiero obtener la velocidad de los puntos en estimulos unisensoriales para sociales/aislados solo
        
all_max_velocities = np.array(all_max_velocities)

plt.figure()
plt.hist(all_max_velocities[all_max_velocities < 20], bins = 40)
plt.xlabel('angular_acceleration') #lo cambié, el original decia 'Max Velocities for Selected Events'. Antes puse velocidad angular.
plt.show()

In [ ]:
# from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN, OPTICS
# from sklearn.mixture import GaussianMixture
# import matplotlib.pyplot as plt
# import numpy as np
# import matplotlib.colors as mcolors

# # # Initialize parameters for each algorithm
# # n_clusters = 5 # Number of clusters for algorithms that require it

# # # # DBSCAN parameters
# # eps = 0.7
# # min_samples = 7

# eps = 5.5
# min_samples = 25

# # # # OPTICS parameters
# # # min_samples_optics = 5
# # # xi = 0.05
# # # min_cluster_size = 0.05

# # # Uncomment one of the following clustering algorithms to use

# # # K-Means
# # #labels = KMeans(n_clusters=n_clusters).fit_predict(concatenated_low_D)

# # # Agglomerative Clustering
# # #labels = AgglomerativeClustering(n_clusters=n_clusters).fit_predict(concatenated_low_D)

# # # Spectral Clustering
# # labels = SpectralClustering(n_clusters=n_clusters).fit_predict(concatenated_low_D)

# # # Gaussian Mixture Model
# # #gmm = GaussianMixture(n_components=n_clusters).fit(concatenated_low_D)
# # #labels = gmm.predict(concatenated_low_D)

# # # DBSCAN
# dbscan = DBSCAN(eps=eps, min_samples=min_samples)
# labels = dbscan.fit_predict(concatenated_low_D)

# # # OPTICS
# # #optics = OPTICS(min_samples=min_samples_optics, xi=xi, min_cluster_size=min_cluster_size)
# # #labels = optics.fit_predict(concatenated_low_D)

# # # Add cluster labels to each trial's 'is_cluster' field
# index_counter = 0
# for stimulus, trial_list in trials.items():
#     for trial in trial_list:
#         within_frame_range = (trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1])
#         num_events = len(trial['is_escape'][within_frame_range])
#         trial['is_cluster'] = labels[index_counter:index_counter + num_events]
#         index_counter += num_events

In [ ]:
# Get a list of distinct colors
colors = plt.cm.rainbow(np.linspace(0, 1, len(np.unique(labels))))

plt.figure(figsize=(12,12))
# Scatter plot
for i, label in enumerate(np.unique(labels)):
    cluster_data = concatenated_low_D[labels == label]
    plt.scatter(cluster_data[:, 0], cluster_data[:, 1], color=colors[i % len(colors)], label=label, alpha=1, s=15)

plt.legend()
#plt.savefig(f"02 - 2D representation of events, clustered by hand.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Assuming 'labels' contains the cluster information for all events

index_counter = 0

for stimulus, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        #within_frame_range = (trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1])
        num_events = len(trial['valid_event_times'])
        trial['event_clusters'] = labels[index_counter:index_counter + num_events]
        index_counter += num_events
        
print(index_counter)

In [ ]:
plt.figure(figsize=(12,8))

# Get a list of distinct colors
colors = plt.cm.rainbow(np.linspace(0, 1, len(np.unique(labels))))

for stimulus, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        within_frame_range = (trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1])
        clusters = np.array(trial['event_clusters'][within_frame_range])
        points = trial['low_D_representation'][within_frame_range]
        
        plt.scatter(points[:, 0], points[:, 1], color=colors[clusters+1], alpha=1, s=30)
plt.show()

In [ ]:
# Get a list of distinct colors
colors = plt.cm.rainbow(np.linspace(0, 1, len(np.unique(labels))))

plt.figure(figsize=(12,12))
# Scatter plot
for i, label in enumerate(np.unique(labels)):
    cluster_data = concatenated_low_D[labels == label]
    plt.scatter(cluster_data[:, 0], cluster_data[:, 1], color=colors[i % len(colors)], label=label, alpha=1, s=15)

plt.legend()
#plt.savefig(f"02 - 2D representation of events, clustered by hand.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Numeric to classic code mapping
numeric_to_classic = {
    '0.0': 'no',  # Represents no stimulus
    '0.1': 'min',
    '0.3': 'med',
    '1.0': 'max'
}

# Visual and auditory classic codes
visual_codes = {value: 'v' + value for key, value in numeric_to_classic.items()}
auditory_codes = {value: 'a' + value for key, value in numeric_to_classic.items()}

# Combining visual and auditory codes for multisensory stimuli
stim_pairs_numeric = sorted(trials.keys())

conversion_dict = {}

for pair in stim_pairs_numeric:
    v_intensity, a_intensity = pair.split('_')
    v_code = visual_codes[numeric_to_classic[v_intensity]]
    a_code = auditory_codes[numeric_to_classic[a_intensity]]
    # For no stimulus case, use "ctrl" instead of combining "no" labels
    if v_intensity == '0.0' and a_intensity == '0.0':
        conversion_dict[pair] = 'ctrl'
    else:
        conversion_dict[pair] = f"{v_code}_{a_code}" if v_intensity != '0.0' and a_intensity != '0.0' else v_code if a_intensity == '0.0' else a_code

print(conversion_dict)

In [ ]:
t = 50

print(trials["1.0_1.0"][t].keys())
print(trials["1.0_1.0"][t]["metadata"].keys())
print(trials["1.0_1.0"][t]["metadata"]["F_stim"])
print(trials["1.0_1.0"][t]["features"].keys())
print(trials["1.0_1.0"][t]["event_times"])
print(trials["1.0_1.0"][t]["valid_event_times"])
print(trials["1.0_1.0"][t]["features"]['head_velocity'].shape)
print(trials["1.0_1.0"][t]["features"]['angular_acceleration'].shape)
print(trials["1.0_1.0"][t]["event_features"].shape)
print(trials["1.0_1.0"][t]["low_D_representation"].shape)
print(trials["1.0_1.0"][t]["event_clusters"])

In [ ]:
all_F_stims = []

for stim in trials:
    for trial in trials[stim]:
        all_F_stims.append(trial["metadata"]["F_stim"])
        
plt.hist(all_F_stims)
plt.show()

print(min(all_F_stims), max(all_F_stims))

In [ ]:
organized_data = {}

relative_stim_window = (-500,500)

frames_before = 8000
frames_after = 4000

for intensity, stim_code in conversion_dict.items():
    print(intensity, stim_code)
    organized_data[stim_code] = {"trials": [], "videos": [], "fish": [], "sequence_in_fish": [], "sequence_in_day": [], 
                                "sequence_in_exp": [], "age": [], "vis_intensity": [], "aud_intensity": [], 
                                "head_velocity": [], "tail_velocity": [], "angular_velocity": [], "sum_tail_angles": [],
                                "tail6_velocity": [], "head_tail_distance": [], "head_acceleration": [], "tail_acceleration": [], 
                                "angular_acceleration": [], "head_jerk": [], "tail_jerk": [], "sum_curvature": [],
                                "angle_velocity_heading": [], "curvature_rate": [], "event_times": [], "valid_event_times": [],
                                "events_in_high_dim": [], "events_in_low_dim": [], "behavior_type": [], "is_escape": []}
    
    for trial in trials[intensity]:
        F_stim = int(trial["metadata"]["F_stim"])
        
        organized_data[stim_code]["trials"].append(int(trial["metadata"]["Trial"]))
        organized_data[stim_code]["videos"].append(trial["metadata"]["Video"])
        organized_data[stim_code]["fish"].append(int(trial["metadata"]["Pez"]))
        organized_data[stim_code]["sequence_in_fish"].append(int(trial["metadata"]["Secuencia"]))
        organized_data[stim_code]["sequence_in_day"].append(int(trial["metadata"]["Sec_día"]))
        organized_data[stim_code]["sequence_in_exp"].append(int(trial["metadata"]["Sec_Exp"]))
        organized_data[stim_code]["age"].append(int(trial["metadata"]["Edad_días"]))
        organized_data[stim_code]["vis_intensity"].append(float(trial["metadata"]["Visual"]))
        organized_data[stim_code]["aud_intensity"].append(float(trial["metadata"]["Auditivo"]))
        
        organized_data[stim_code]["head_velocity"].append(trial["features"]['head_velocity'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["tail_velocity"].append(trial["features"]['tail_velocity'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["angular_velocity"].append(trial["features"]['angular_velocity'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["sum_tail_angles"].append(trial["features"]['sum_tail_angles'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["tail6_velocity"].append(trial["features"]['tail6_velocity'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["head_tail_distance"].append(trial["features"]['head_tail_distance'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["head_acceleration"].append(trial["features"]['head_acceleration'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["tail_acceleration"].append(trial["features"]['tail_acceleration'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["angular_acceleration"].append(trial["features"]['angular_acceleration'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["head_jerk"].append(trial["features"]['head_jerk'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["tail_jerk"].append(trial["features"]['tail_jerk'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["sum_curvature"].append(trial["features"]['sum_curvature'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["angle_velocity_heading"].append(trial["features"]['angle_velocity_heading'][F_stim-frames_before:F_stim+frames_after])
        organized_data[stim_code]["curvature_rate"].append(trial["features"]['curvature_rate'][F_stim-frames_before:F_stim+frames_after])
        
        event_times = np.array(trial["event_times"])
        event_times = event_times[(event_times > (F_stim-frames_before)) & (event_times < (F_stim+frames_after))]
        event_times = event_times - (F_stim-frames_before)
        organized_data[stim_code]["event_times"].append(event_times)
        
        valid_event_times = np.array(trial["valid_event_times"])
        valid_event_times = valid_event_times[(valid_event_times > (F_stim-frames_before)) & (valid_event_times < (F_stim+frames_after))]
        valid_event_times = valid_event_times - (F_stim-frames_before)
        organized_data[stim_code]["valid_event_times"].append(valid_event_times)
        
        valid_event_times = np.array(trial["valid_event_times"])
        organized_data[stim_code]["events_in_high_dim"].append(trial["event_features"][(valid_event_times > (F_stim-frames_before)) & (valid_event_times < (F_stim+frames_after))])
        
        if "low_D_representation" in trial: 
            organized_data[stim_code]["events_in_low_dim"].append(trial["low_D_representation"][(valid_event_times > (F_stim-frames_before)) & (valid_event_times < (F_stim+frames_after))])
            organized_data[stim_code]["behavior_type"].append(np.array(trial["event_clusters"][(valid_event_times > (F_stim-frames_before)) & (valid_event_times < (F_stim+frames_after))]))
        else:
            organized_data[stim_code]["events_in_low_dim"].append(np.zeros((0,2)))
            organized_data[stim_code]["behavior_type"].append(np.zeros((0,)))
        
        valid_event_times = np.array(trial["valid_event_times"])
        valid_event_times = valid_event_times[(valid_event_times > (F_stim+relative_stim_window[0])) & (valid_event_times < (F_stim+relative_stim_window[1]))]
        organized_data[stim_code]["is_escape"].append(len(valid_event_times) > 0)
    
    for key in organized_data[stim_code].keys():
        if key not in ["videos", "event_times", "valid_event_times", "events_in_high_dim", "events_in_low_dim", "behavior_type"]:
            organized_data[stim_code][key] = np.array(organized_data[stim_code][key])
            print(key, organized_data[stim_code][key].shape)
        else:
            print(key, len(organized_data[stim_code][key]))
            
    print()

organized_data["stim_time"] = frames_before
organized_data["stim_window"] = (frames_before+relative_stim_window[0], frames_before+relative_stim_window[1])

organized_data["colors"] = {"ctrl":(0.5,0.5,0.5),
                            "vmin":(1,0.8,0.8),
                            "vmed":(1,0.4,0.4),
                            "vmax":(1,0,0),
                            "amin":(0.8,0.8,1),
                            "amed":(0.4,0.4,1),
                            "amax":(0,0,1),
                            "vmin_amin":(1,0.8,1),
                            "vmed_amin":(1,0.5,0.85),
                            "vmax_amin":(1,0.3,0.7),
                            "vmin_amed":(0.8,0.6,1),
                            "vmed_amed":(0.9,0.375,0.925),
                            "vmax_amed":(1,0.15,0.85),
                            "vmin_amax":(0.6,0.4,1),
                            "vmed_amax":(0.8,0.2,1),
                            "vmax_amax":(1,0,1)}

# Create the stim_codes dictionary
organized_data["stim_labels"] = {"ctrl":"Control",
                                "vmin":"Vis Min",
                                "vmed":"Vis Med",
                                "vmax":"Vis Max",
                                "amin":"Aud Min",
                                "amed":"Aud Med",
                                "amax":"Aud Max",
                                "vmin_amin":"Vis Min + Aud Min",
                                "vmed_amin":"Vis Med + Aud Min",
                                "vmax_amin":"Vis Max + Aud Min",
                                "vmin_amed":"Vis Min + Aud Med",
                                "vmed_amed":"Vis Med + Aud Med",
                                "vmax_amed":"Vis Max + Aud Med",
                                "vmin_amax":"Vis Min + Aud Max",
                                "vmed_amax":"Vis Min + Aud Max",
                                "vmax_amax":"Vis Max + Aud Max"}

In [ ]:
data = organized_data
i = 5
for stim in data:
    if "max" in stim or "med" in stim or "min" in stim:
        print(stim, len(data[stim]["valid_event_times"]), data[stim]["valid_event_times"][i].shape, len(data[stim]["behavior_type"]), data[stim]["behavior_type"][i].shape)

In [ ]:
import pickle

os.chdir(r"D:\Videos Zebrafish\ConfocalLike3 (Febrero 2024)")

# Writing the list to a file using Pickle
with open('organized_data_behavior_2024_new_tracking.pkl', 'wb') as file:
    pickle.dump(organized_data, file)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def show_color(rgb):
    """
    Displays a color in a big square based on RGB values.

    Parameters:
    rgb (tuple): A tuple of three numbers between 0 and 1, representing the RGB values.
    """
    # Create a figure and a subplot
    fig, ax = plt.subplots()
    # Create a rectangle patch with the given RGB color
    rect = patches.Rectangle((0.1, 0.1), 1.0, 1.0, linewidth=1, edgecolor='black', facecolor=rgb)
    # Add the rectangle patch to the subplot
    ax.add_patch(rect)
    # Set the limits of the axes
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')  # Turn off the axis
    plt.show()  # Display the figure

colors = {"ctrl":(0.5,0.5,0.5),
            "vmin":(1,0.8,0.8),
            "vmed":(1,0.4,0.4),
            "vmax":(1,0,0),
            "amin":(0.8,0.8,1),
            "amed":(0.4,0.4,1),
            "amax":(0,0,1),
            "vmin_amin":(1,0.8,1),
            "vmed_amin":(1,0.5,0.85),
            "vmax_amin":(1,0.3,0.7),
            "vmin_amed":(0.8,0.6,1),
            "vmed_amed":(0.9,0.375,0.925),
            "vmax_amed":(1,0.15,0.85),
            "vmin_amax":(0.6,0.4,1),
            "vmed_amax":(0.8,0.2,1),
            "vmax_amax":(1,0,1)}

for key in colors:
    show_color(colors[key])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming organized_data is your main data structure

# Step 1: Calculate response probability per stim type and age group
response_probabilities = {}  # {stim_type: {age: probability, ...}, ...}

stim_types = ['ctrl', 'amin', 'amed', 'amax', 'vmin', 'vmin_amin', 'vmin_amed', 'vmin_amax', 'vmed', 'vmed_amin', 'vmed_amed', 'vmed_amax', 'vmax', 'vmax_amin', 'vmax_amed', 'vmax_amax']

for stim_type in stim_types:
    # Extract age and is_escape data
    ages = organized_data[stim_type]["age"]
    escapes = organized_data[stim_type]["is_escape"]
    
    # Group by age and calculate response probability
    age_groups = np.unique(ages)
    stim_response_probabilities = {}
    
    for age in age_groups:
        is_escape_at_age = escapes[ages == age]
        probability = np.mean(is_escape_at_age)
        stim_response_probabilities[age] = probability
    
    response_probabilities[stim_type] = stim_response_probabilities

# Step 2: Prepare the data for plotting - done during plotting to keep the structure simple

# Step 3: Plot
plt.figure(figsize=(10, 8))

for stim_type, age_probs in response_probabilities.items():
    ages = list(age_probs.keys())
    probabilities = list(age_probs.values())
    plt.plot(ages, probabilities, label=organized_data["stim_labels"][stim_type], color=organized_data["colors"][stim_type])

plt.xlabel('Age')
plt.ylabel('Response Probability')
plt.title('Response Probability per Stim Type and Age')
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()


In [ ]:
# import random

# cluster_events = {}

# for cluster in np.unique(labels):
#     cluster_events[cluster] = []

# for stimulus, trial_list in trials.items():
#     for trial in trial_list:
#         for i, cluster in enumerate(trial['event_clusters']):
#             event_time = trial['valid_event_times'][i]
#             cluster_events[cluster].append((trial['video_name'], event_time))

# # Randomly sample 10 events from each cluster
# for cluster, events in cluster_events.items():
#     cluster_events[cluster] = random.sample(events, min(10, len(events)))


In [ ]:
import random

cluster_events = {}

for cluster in np.unique(labels):
    cluster_events[cluster] = []

for stimulus, trial_list in trials.items():
    for trial in trial_list:
        for i, cluster in enumerate(trial['event_clusters']):
            event_time = trial['valid_event_times'][i]
            cluster_events[cluster].append((trial['video_name'], event_time, trial['event_features_over_time'][i]))

# Randomly sample 10 events from each cluster except for cluster 1, from which we'll sample 30 events
for cluster, events in cluster_events.items():
    if cluster == -1:
        cluster_events[cluster] = random.sample(events, min(1, len(events)))
    else:
        cluster_events[cluster] = random.sample(events, min(10, len(events)))


In [ ]:
# import numpy as np

# all_low_D_representations = []
# all_is_escape = []

# # Assume `trials` is your dictionary containing all the data
# for stimulus_type, trial_list in trials.items():
#     for trial in trial_list:
#         if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
#             continue
        
#         all_low_D_representations.append(trial['low_D_representation'])
#         all_is_escape.append(trial['is_escape'])

# all_low_D_representations = np.vstack(all_low_D_representations)
# all_is_escape = np.concatenate(all_is_escape)

In [ ]:
# from sklearn.neighbors import KernelDensity

# # Prepare the data
# escape_points = all_low_D_representations[all_is_escape]
# non_escape_points = all_low_D_representations[~all_is_escape]

# # Perform KDE
# kde_escape = KernelDensity(kernel='gaussian', bandwidth=0.35).fit(escape_points)
# kde_non_escape = KernelDensity(kernel='gaussian', bandwidth=0.35).fit(non_escape_points)

In [ ]:
# # Generate a 2D grid over which to evaluate the KDE
# x_vals = np.linspace(min(all_low_D_representations[:, 0]), max(all_low_D_representations[:, 0]), 100)
# y_vals = np.linspace(min(all_low_D_representations[:, 1]), max(all_low_D_representations[:, 1]), 100)
# xx, yy = np.meshgrid(x_vals, y_vals)

# grid_points = np.c_[xx.ravel(), yy.ravel()]

# # Evaluate KDE at grid points
# pdf_escape = np.exp(kde_escape.score_samples(grid_points))
# pdf_non_escape = np.exp(kde_non_escape.score_samples(grid_points))

# # Calculate ratio
# pdf_ratio = pdf_escape / (pdf_escape + pdf_non_escape + 1e-10)  # Adding a small constant to avoid division by zero
# pdf_ratio = pdf_ratio.reshape(xx.shape)

In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure(data=[go.Surface(z=pdf_ratio, x=xx, y=yy)])
# fig.update_layout(scene=dict(zaxis=dict(range=[0, np.max(pdf_ratio)])))
# fig.show()


In [ ]:
# from scipy.interpolate import interp2d

# threshold = 0.35

# above_threshold = pdf_ratio > threshold

# # Create an interpolation function based on the calculated ratio
# interp_func = interp2d(x_vals, y_vals, pdf_ratio, kind='linear')

# # Evaluate the function at the points in all_low_D_representations
# evaluated_ratios = np.array([interp_func(x, y)[0] for x, y in all_low_D_representations])

# # Create estimated_escape array based on the threshold
# estimated_escape = (evaluated_ratios > threshold).astype(int)

# print(estimated_escape.sum())

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12,12))
# plt.scatter(all_low_D_representations[:, 0], all_low_D_representations[:, 1], c=estimated_escape, cmap='viridis', s=10)
# plt.colorbar(label='Estimated Escape')
# plt.xlabel('Dimension 1')
# plt.ylabel('Dimension 2')
# plt.show()

In [ ]:
# index_start = 0

# for stimulus_type, trial_list in trials.items():
#     for trial in trial_list:
#         if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
#             continue
            
#         num_events = trial['low_D_representation'].shape[0]
#         trial['estimated_escape'] = estimated_escape[index_start:index_start + num_events]
#         trial['estimated_escape'] = trial['estimated_escape'].astype('bool')
#         index_start += num_events

In [ ]:
# index_start = 0

# for stimulus_type, trial_list in trials.items():
#     for trial in trial_list:
#         if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
#             continue
            
#         clust = trial['is_cluster']
        
#         fast_escape = np.array([clust == 0, clust == 1, clust == 5, clust == 6, clust == 9]).any(axis=0)
        
#         trial['estimated_escape'] = fast_escape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu


# Initialize lists to store the values for each parameter
params_cluster_0 = {
    'max_head_velocity': [],
    'max_tail_velocity': [],
    'max_abs_angular_velocity': [],
    'min_head_tail_distance': [],
    'max_abs_angular_acceleration': [],
    'max_abs_curvature': [],
    'max_abs_curvature_rate': [],
    'max_abs_head_jerk': []
}

params_other_clusters = {
    'max_head_velocity': [],
    'max_tail_velocity': [],
    'max_abs_angular_velocity': [],
    'min_head_tail_distance': [],
    'max_abs_angular_acceleration': [],
    'max_abs_curvature': [],
    'max_abs_curvature_rate': [],
    'max_abs_head_jerk': []
}

bonferroni_alpha = 0.05 / 8

# Iterate over each trial and its events
for stim_type, trials_i in trials.items():
    for trial in trials_i:
        event_clusters = trial['event_clusters']
        valid_event_times = trial['valid_event_times']
        features = trial['features']

        for idx, event_time in enumerate(valid_event_times):
            cluster = event_clusters[idx]

            # Determine the event window
            start_frame = event_time - 5
            end_frame = event_time + 25

            # Extract the relevant features within the event window
            head_velocity = features['head_velocity'][start_frame:end_frame]
            tail_velocity = features['tail_velocity'][start_frame:end_frame]
            angular_velocity = features['angular_velocity'][start_frame:end_frame]
            head_tail_distance = features['head_tail_distance'][start_frame:end_frame]
            angular_acceleration = features['angular_acceleration'][start_frame:end_frame]
            curvature = features['sum_curvature'][start_frame:end_frame]
            curvature_rate = features['curvature_rate'][start_frame:end_frame]
            head_jerk = features['head_jerk'][start_frame:end_frame]

            # Compute the representative parameters for the event
            params = {
                'max_head_velocity': np.max(head_velocity),
                'max_tail_velocity': np.max(tail_velocity),
                'max_abs_angular_velocity': np.max(np.abs(angular_velocity)),
                'min_head_tail_distance': np.min(head_tail_distance),
                'max_abs_angular_acceleration': np.max(np.abs(angular_acceleration)),
                'max_abs_curvature': np.max(np.abs(curvature)),
                'max_abs_curvature_rate': np.max(np.abs(curvature_rate)),
                'max_abs_head_jerk': np.max(np.abs(head_jerk))
            }

            # Store the parameters in the appropriate list
            if cluster == 0:
                for key in params.keys():
                    params_cluster_0[key].append(params[key])
            else:
                for key in params.keys():
                    params_other_clusters[key].append(params[key])

# Now plot the mirrored histograms
plt.rcParams["figure.figsize"] = (12, 12)
fig, axes = plt.subplots(4, 2)

for idx, param_name in enumerate(params_cluster_0.keys()):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]

    # Calculate min and max values for both cluster 0 and other clusters
    min_value = min(np.percentile(params_cluster_0[param_name],5), np.percentile(params_other_clusters[param_name],5))
    max_value = max(np.percentile(params_cluster_0[param_name],95), np.percentile(params_other_clusters[param_name],95))
    
    n_bins = 15
    
    # Define the bin edges
    bin_edges = np.linspace(min_value, max_value, n_bins+1)
    
    # Plot histogram for cluster 0
    sns.histplot(x=params_cluster_0[param_name], stat="density", bins=bin_edges, edgecolor='black', ax=ax)
    
    # Plot histogram for other clusters
    heights, bins = np.histogram(params_other_clusters[param_name], density=True, bins=bin_edges)
    heights *= -1  # Multiply by -1 to reverse
    bin_width = np.diff(bins)[0]
    bin_pos = bins[:-1] + bin_width / 2  # Keep these positive
    ax.bar(bin_pos, heights, width=bin_width, edgecolor='black')
    
    # Perform the Mann-Whitney U test
    stat, p_value = mannwhitneyu(params_cluster_0[param_name], params_other_clusters[param_name], alternative='two-sided')
    
    # Add the p-value to the plot
    ax.text(0.1, 0.9, f'p = {p_value:.3f}', transform=ax.transAxes)
    
    if p_value < bonferroni_alpha:
        ax.text(0.2, 0.75, '*', transform=ax.transAxes, fontsize=20)
    
    ax.set_title(param_name)

#plt.savefig(f"03 - Difference in feature distribution between C-starts and other escape events.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

stim_window = (10000,11000)
#stim_window = (9500,11500)

desired_cluster = 1

# Iterate through the trials to set 'is_estimated_escape'
index_start = 0
for stimulus_type, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        within_frame_range = (trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1]) & (trial['event_clusters'] == desired_cluster)
        #trial_estimated_escapes = trial['estimated_escape'][within_frame_range]
        
        #trial['is_estimated_escape'] = any(trial_estimated_escapes)
        event_clusters = trial['event_clusters']
        trial['is_estimated_escape'] = within_frame_range.sum() > 0

# Initialize a dictionary to hold escape probabilities for each stimulus type
escape_probabilities = {}

for stimulus_type, trial_list in trials.items():
    is_estimated_escape_values = []
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        is_estimated_escape_values.append(trial['is_estimated_escape'])
    escape_probabilities[stimulus_type] = np.mean(is_estimated_escape_values)

# Prepare data for heatmap
visual_intensities = []
auditory_intensities = []
probabilities = []

for stimulus_type, probability in escape_probabilities.items():
    visual_intensity, auditory_intensity = map(float, stimulus_type.split('_'))
    visual_intensities.append(visual_intensity)
    auditory_intensities.append(auditory_intensity)
    probabilities.append(probability)

import matplotlib.pyplot as plt
import seaborn as sns

# Create a DataFrame for easier plotting
import pandas as pd
df = pd.DataFrame({
    'Visual': visual_intensities,
    'Auditory': auditory_intensities,
    'Probability': probabilities
})

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df.pivot("Visual", "Auditory", "Probability"), cmap="Oranges", annot=True, cbar_kws={'label': 'Escape Probability'}, vmin=0, vmax=1)
plt.gca().invert_yaxis()
plt.title("Escape Probability Heatmap")
#plt.savefig(f"04 - Observed probability matrix.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

# Function to calculate expected probability
def calc_expected_p(p_v, p_a):
    return p_v + p_a - p_v * p_a

# Function to calculate integration coefficient
def calc_integration(p_observed, p_expected):
    return (p_observed - p_expected) / (p_observed + p_expected)

# Initialize an empty DataFrame for storing integration coefficients
integration_df = pd.DataFrame(columns=['Visual', 'Auditory', 'Integration'])

# Iterate through each multisensory stimulus
for _, row in df[(df['Visual'] != 0) & (df['Auditory'] != 0)].iterrows():
    visual_intensity = row['Visual']
    auditory_intensity = row['Auditory']
    p_observed = row['Probability']
    
    # Get the probabilities of the associated unisensory stimuli
    p_v = df[(df['Visual'] == visual_intensity) & (df['Auditory'] == 0)]['Probability'].values[0]
    p_a = df[(df['Visual'] == 0) & (df['Auditory'] == auditory_intensity)]['Probability'].values[0]
    
    # Calculate expected probability
    p_expected = calc_expected_p(p_v, p_a)
    
    # Calculate integration coefficient
    integration = calc_integration(p_observed, p_expected)
    
    # Append to the DataFrame
    integration_df = integration_df.append({'Visual': visual_intensity, 'Auditory': auditory_intensity, 'Integration': integration}, ignore_index=True)


# Pivot the DataFrame to get a matrix form suitable for heatmap
integration_matrix = integration_df.pivot("Visual", "Auditory", "Integration")

# Create the heatmap
plt.figure(figsize=(10, 8))
heatmap = sns.heatmap(integration_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
heatmap.set_title('Integration Coefficients Heatmap')
plt.gca().invert_yaxis()  # Invert y-axis
plt.show()

# Initialize an empty DataFrame for storing integration coefficients
integration_df = pd.DataFrame(columns=['Visual', 'Auditory', 'Integration'])

# Iterate through each multisensory stimulus
for _, row in df[(df['Visual'] != 0) & (df['Auditory'] != 0)].iterrows():
    visual_intensity = row['Visual']
    auditory_intensity = row['Auditory']
    p_observed = row['Probability']
    
    # Get the probabilities of the associated unisensory stimuli
    p_v = df[(df['Visual'] == visual_intensity) & (df['Auditory'] == 0)]['Probability'].values[0]
    p_a = df[(df['Visual'] == 0) & (df['Auditory'] == auditory_intensity)]['Probability'].values[0]
    
    # Calculate expected probability
    p_expected = max(p_v, p_a)
    
    # Calculate integration coefficient
    integration = calc_integration(p_observed, p_expected)
    
    # Append to the DataFrame
    integration_df = integration_df.append({'Visual': visual_intensity, 'Auditory': auditory_intensity, 'Integration': integration}, ignore_index=True)


# Pivot the DataFrame to get a matrix form suitable for heatmap
integration_matrix = integration_df.pivot("Visual", "Auditory", "Integration")

# Create the heatmap
plt.figure(figsize=(10, 8))
heatmap = sns.heatmap(integration_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
heatmap.set_title('Integration Coefficients Heatmap')
plt.gca().invert_yaxis()  # Invert y-axis
plt.show()

In [ ]:
desired_cluster = 0

# Iterate through the trials to set 'is_estimated_escape'
index_start = 0
for stimulus_type, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        within_frame_range = (trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1]) & (trial['event_clusters'] == desired_cluster)
        #trial_estimated_escapes = trial['estimated_escape'][within_frame_range]
        
        #trial['is_estimated_escape'] = any(trial_estimated_escapes)
        event_clusters = trial['event_clusters']
        trial['is_estimated_escape'] = within_frame_range.sum() > 0

# Initialize a dictionary to hold escape probabilities for each stimulus type
escape_probabilities = {}

for stimulus_type, trial_list in trials.items():
    is_estimated_escape_values = []
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        is_estimated_escape_values.append(trial['is_estimated_escape'])
    escape_probabilities[stimulus_type] = np.mean(is_estimated_escape_values)

# Prepare data for heatmap
visual_intensities = []
auditory_intensities = []
probabilities = []

for stimulus_type, probability in escape_probabilities.items():
    visual_intensity, auditory_intensity = map(float, stimulus_type.split('_'))
    visual_intensities.append(visual_intensity)
    auditory_intensities.append(auditory_intensity)
    probabilities.append(probability)

import matplotlib.pyplot as plt
import seaborn as sns

# Create a DataFrame for easier plotting
import pandas as pd
df = pd.DataFrame({
    'Visual': visual_intensities,
    'Auditory': auditory_intensities,
    'Probability': probabilities
})

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df.pivot("Visual", "Auditory", "Probability"), cmap="Oranges", annot=True, cbar_kws={'label': 'Escape Probability'}, vmin=0, vmax=1)
plt.gca().invert_yaxis()
plt.title("Escape Probability Heatmap")
#plt.savefig(f"04 - Observed probability matrix.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

# Function to calculate expected probability
def calc_expected_p(p_v, p_a):
    return p_v + p_a - p_v * p_a

# Function to calculate integration coefficient
def calc_integration(p_observed, p_expected):
    return (p_observed - p_expected) / (p_observed + p_expected)

# Initialize an empty DataFrame for storing integration coefficients
integration_df = pd.DataFrame(columns=['Visual', 'Auditory', 'Integration'])

# Iterate through each multisensory stimulus
for _, row in df[(df['Visual'] != 0) & (df['Auditory'] != 0)].iterrows():
    visual_intensity = row['Visual']
    auditory_intensity = row['Auditory']
    p_observed = row['Probability']
    
    # Get the probabilities of the associated unisensory stimuli
    p_v = df[(df['Visual'] == visual_intensity) & (df['Auditory'] == 0)]['Probability'].values[0]
    p_a = df[(df['Visual'] == 0) & (df['Auditory'] == auditory_intensity)]['Probability'].values[0]
    
    # Calculate expected probability
    p_expected = calc_expected_p(p_v, p_a)
    
    # Calculate integration coefficient
    integration = calc_integration(p_observed, p_expected)
    
    # Append to the DataFrame
    integration_df = integration_df.append({'Visual': visual_intensity, 'Auditory': auditory_intensity, 'Integration': integration}, ignore_index=True)


# Pivot the DataFrame to get a matrix form suitable for heatmap
integration_matrix = integration_df.pivot("Visual", "Auditory", "Integration")

# Create the heatmap
plt.figure(figsize=(10, 8))
heatmap = sns.heatmap(integration_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
heatmap.set_title('Integration Coefficients Heatmap')
plt.gca().invert_yaxis()  # Invert y-axis
plt.show()

# Initialize an empty DataFrame for storing integration coefficients
integration_df = pd.DataFrame(columns=['Visual', 'Auditory', 'Integration'])

# Iterate through each multisensory stimulus
for _, row in df[(df['Visual'] != 0) & (df['Auditory'] != 0)].iterrows():
    visual_intensity = row['Visual']
    auditory_intensity = row['Auditory']
    p_observed = row['Probability']
    
    # Get the probabilities of the associated unisensory stimuli
    p_v = df[(df['Visual'] == visual_intensity) & (df['Auditory'] == 0)]['Probability'].values[0]
    p_a = df[(df['Visual'] == 0) & (df['Auditory'] == auditory_intensity)]['Probability'].values[0]
    
    # Calculate expected probability
    p_expected = max(p_v, p_a)
    
    # Calculate integration coefficient
    integration = calc_integration(p_observed, p_expected)
    
    # Append to the DataFrame
    integration_df = integration_df.append({'Visual': visual_intensity, 'Auditory': auditory_intensity, 'Integration': integration}, ignore_index=True)


# Pivot the DataFrame to get a matrix form suitable for heatmap
integration_matrix = integration_df.pivot("Visual", "Auditory", "Integration")

# Create the heatmap
plt.figure(figsize=(10, 8))
heatmap = sns.heatmap(integration_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
heatmap.set_title('Integration Coefficients Heatmap')
plt.gca().invert_yaxis()  # Invert y-axis
plt.show()

In [ ]:
# Iterate through the trials to set 'is_estimated_escape'
index_start = 0
for stimulus_type, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        within_frame_range = (trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1])
        #trial_estimated_escapes = trial['estimated_escape'][within_frame_range]
        
        #trial['is_estimated_escape'] = any(trial_estimated_escapes)
        trial['is_estimated_escape'] = within_frame_range.sum() > 0

# Initialize a dictionary to hold escape probabilities for each stimulus type
escape_probabilities = {}

for stimulus_type, trial_list in trials.items():
    is_estimated_escape_values = []
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        is_estimated_escape_values.append(trial['is_estimated_escape'])
    escape_probabilities[stimulus_type] = np.mean(is_estimated_escape_values)

# Prepare data for heatmap
visual_intensities = []
auditory_intensities = []
probabilities = []

for stimulus_type, probability in escape_probabilities.items():
    visual_intensity, auditory_intensity = map(float, stimulus_type.split('_'))
    visual_intensities.append(visual_intensity)
    auditory_intensities.append(auditory_intensity)
    probabilities.append(probability)

import matplotlib.pyplot as plt
import seaborn as sns

# Create a DataFrame for easier plotting
import pandas as pd
df = pd.DataFrame({
    'Visual': visual_intensities,
    'Auditory': auditory_intensities,
    'Probability': probabilities
})

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df.pivot("Visual", "Auditory", "Probability"), cmap="Oranges", annot=True, cbar_kws={'label': 'Escape Probability'}, vmin=0, vmax=1)
plt.gca().invert_yaxis()
plt.title("Escape Probability Heatmap")
#plt.savefig(f"04 - Observed probability matrix.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

# Function to calculate expected probability
def calc_expected_p(p_v, p_a):
    return p_v + p_a - p_v * p_a

# Function to calculate integration coefficient
def calc_integration(p_observed, p_expected):
    return (p_observed - p_expected) / (p_observed + p_expected)

# Initialize an empty DataFrame for storing integration coefficients
integration_df = pd.DataFrame(columns=['Visual', 'Auditory', 'Integration'])

# Iterate through each multisensory stimulus
for _, row in df[(df['Visual'] != 0) & (df['Auditory'] != 0)].iterrows():
    visual_intensity = row['Visual']
    auditory_intensity = row['Auditory']
    p_observed = row['Probability']
    
    # Get the probabilities of the associated unisensory stimuli
    p_v = df[(df['Visual'] == visual_intensity) & (df['Auditory'] == 0)]['Probability'].values[0]
    p_a = df[(df['Visual'] == 0) & (df['Auditory'] == auditory_intensity)]['Probability'].values[0]
    
    # Calculate expected probability
    p_expected = calc_expected_p(p_v, p_a)
    
    # Calculate integration coefficient
    integration = calc_integration(p_observed, p_expected)
    
    # Append to the DataFrame
    integration_df = integration_df.append({'Visual': visual_intensity, 'Auditory': auditory_intensity, 'Integration': integration}, ignore_index=True)


# Pivot the DataFrame to get a matrix form suitable for heatmap
integration_matrix = integration_df.pivot("Visual", "Auditory", "Integration")

# Create the heatmap
plt.figure(figsize=(10, 8))
heatmap = sns.heatmap(integration_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
heatmap.set_title('Integration Coefficients Heatmap')
plt.gca().invert_yaxis()  # Invert y-axis
plt.show()

# Initialize an empty DataFrame for storing integration coefficients
integration_df = pd.DataFrame(columns=['Visual', 'Auditory', 'Integration'])

# Iterate through each multisensory stimulus
for _, row in df[(df['Visual'] != 0) & (df['Auditory'] != 0)].iterrows():
    visual_intensity = row['Visual']
    auditory_intensity = row['Auditory']
    p_observed = row['Probability']
    
    # Get the probabilities of the associated unisensory stimuli
    p_v = df[(df['Visual'] == visual_intensity) & (df['Auditory'] == 0)]['Probability'].values[0]
    p_a = df[(df['Visual'] == 0) & (df['Auditory'] == auditory_intensity)]['Probability'].values[0]
    
    # Calculate expected probability
    p_expected = max(p_v, p_a)
    
    # Calculate integration coefficient
    integration = calc_integration(p_observed, p_expected)
    
    # Append to the DataFrame
    integration_df = integration_df.append({'Visual': visual_intensity, 'Auditory': auditory_intensity, 'Integration': integration}, ignore_index=True)


# Pivot the DataFrame to get a matrix form suitable for heatmap
integration_matrix = integration_df.pivot("Visual", "Auditory", "Integration")

# Create the heatmap
plt.figure(figsize=(10, 8))
heatmap = sns.heatmap(integration_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
heatmap.set_title('Integration Coefficients Heatmap')
plt.gca().invert_yaxis()  # Invert y-axis
plt.show()

In [ ]:
# Define custom colors based on visual intensity
custom_colors = {
    "0.0_0.0": (0.3, 0.3, 0.3),
    "0.0_1.0": (0., 0.2        , 1.        ),
    "0.3_0.0": (1.        , 0.72156863, 0.16862745),
    "0.0_0.3": (0.        , 0.83529412, 1.        ),
    "1.0_0.0": (1., 0., 0.),
    
    "0.3_0.3": (0.83529412, 0.56862745, 1),
    "0.3_1.0": (0.57254902, 0.01176471, 1),
    "1.0_0.3": (1.        , 0.56862745, 0.75686275),
    "1.0_1.0": (1.        , 0.        , 0.81568627)
}

custom_labels = {
    "0.0_0.0": "Control",
    "0.0_1.0": "Aud Max",
    "0.3_0.0": "Vis Min",
    "0.0_0.3": "Aud Min",
    "1.0_0.0": "Vis Max",
    
    "0.3_0.3": "Vis Min + Aud Min",
    "0.3_1.0": "Vis Min + Aud Max",
    "1.0_0.3": "Vis Max + Aud Min",
    "1.0_1.0": "Vis Max + Aud Max"
}

In [ ]:
# Iterate through the trials to set 'is_estimated_escape'
index_start = 0
for stimulus_type, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        within_frame_range = (trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1])
        #trial_estimated_escapes = trial['estimated_escape'][within_frame_range]
        
        #trial['is_estimated_escape'] = any(trial_estimated_escapes)
        trial['is_estimated_escape'] = within_frame_range.sum() > 0

# Initialize a dictionary to hold escape probabilities for each stimulus type
escape_probabilities = {}

for stimulus_type, trial_list in trials.items():
    is_estimated_escape_values = []
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        is_estimated_escape_values.append(trial['is_estimated_escape'])
    escape_probabilities[stimulus_type] = np.mean(is_estimated_escape_values)

# Prepare data for heatmap
visual_intensities = []
auditory_intensities = []
probabilities = []

for stimulus_type, probability in escape_probabilities.items():
    visual_intensity, auditory_intensity = map(float, stimulus_type.split('_'))
    visual_intensities.append(visual_intensity)
    auditory_intensities.append(auditory_intensity)
    probabilities.append(probability)

import matplotlib.pyplot as plt
import seaborn as sns

# Create a DataFrame for easier plotting
import pandas as pd
df = pd.DataFrame({
    'Visual': visual_intensities,
    'Auditory': auditory_intensities,
    'Probability': probabilities
})

df['CustomColor'] = df.apply(lambda row: custom_colors[f"{row['Visual']}_{row['Auditory']}"], axis=1)

# Create the heatmap
plt.figure(figsize=(8, 8))
heatmap = sns.heatmap(df.pivot("Visual", "Auditory", "Probability"), annot_kws={"size": 24}, cmap="Oranges", cbar=False,annot=True, cbar_kws={'label': 'Escape Probability'}, vmin=0, vmax=1)

# Sort unique intensities
sorted_visual = sorted(df['Visual'].unique())
sorted_auditory = sorted(df['Auditory'].unique())

ax = plt.gca()  # Get the current Axes instance

for i, vis in enumerate(sorted_visual):  # Reverse to match the inverted y-axis
    for j, aud in enumerate(sorted_auditory):
        color = custom_colors.get(f"{vis}_{aud}", "white")
        ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=True, color=color))

plt.gca().invert_yaxis()

# Set the fontsize for axis ticks
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=14)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=14)

# Set the fontsize for axis labels and title
heatmap.set_xlabel("Auditory", fontsize=16)
heatmap.set_ylabel("Visual", fontsize=16)
#heatmap.set_title("Escape Probability Heatmap", fontsize=16)

plt.savefig(f"04 - Observed probability matrix.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Initialize dictionaries to hold escape probabilities and standard errors for each stimulus type
escape_probabilities = {}
escape_standard_errors = {}

for stimulus_type, trial_list in trials.items():
    is_estimated_escape_values = []
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        is_estimated_escape_values.append(trial['is_estimated_escape'])
    
    # Compute mean escape probability
    escape_probabilities[stimulus_type] = np.mean(is_estimated_escape_values)
    
    # Compute standard error
    std_dev = np.std(is_estimated_escape_values)
    sample_size = len(is_estimated_escape_values)
    escape_standard_errors[stimulus_type] = std_dev / np.sqrt(sample_size)

# Prepare data for plotting
stimuli_types = list(escape_probabilities.keys())
prob_values = list(escape_probabilities.values())
error_values = list(escape_standard_errors.values())

# Assuming you have a dictionary called "custom_colors" for coloring bars
colors = [custom_colors[stim] for stim in stimuli_types]

plt.figure(figsize=(10, 6))
plt.bar(stimuli_types, prob_values, color=colors, yerr=error_values, capsize=5)

# Setting labels and title
plt.xlabel('Stimulus Type', fontsize=16)
plt.ylabel('Escape Probability', fontsize=16)
plt.title('Escape Probability by Stimulus Type', fontsize=18)

# Optionally, rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Save the plot
plt.savefig("Escape_Probability_Bar_Chart_with_SE.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
# Iterate through the trials to set 'is_estimated_escape'
index_start = 0
for stimulus_type, trial_list in trials.items():
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        within_frame_range = (trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1])
        #trial_estimated_escapes = trial['estimated_escape'][within_frame_range]
        
        #trial['is_estimated_escape'] = any(trial_estimated_escapes)
        trial['is_estimated_escape'] = within_frame_range.sum() > 0
        
from scipy.stats import chi2_contingency
import numpy as np

# Initialize lists to collect 'is_estimated_escape' values
sat_escape = []
no_sat_escape = []

# Loop through all trials
for stimulus_type, trial_list in trials.items():
    for trial in trial_list:
        if 'is_estimated_escape' in trial:
            video_name = trial['metadata']['Video']
            is_estimated_escape = trial['is_estimated_escape']

            # Categorize the trial based on 'sat' in the video name
            if "sat" in video_name:
                sat_escape.append(is_estimated_escape)
            else:
                no_sat_escape.append(is_estimated_escape)

# Convert lists to numpy arrays for easier manipulation
sat_escape = np.array(sat_escape)
no_sat_escape = np.array(no_sat_escape)

# Calculate the number of escapes and non-escapes for each group
sat_escape_count = np.sum(sat_escape)
sat_no_escape_count = len(sat_escape) - sat_escape_count

no_sat_escape_count = np.sum(no_sat_escape)
no_sat_no_escape_count = len(no_sat_escape) - no_sat_escape_count

# Prepare contingency table
contingency_table = [[sat_escape_count, sat_no_escape_count],
                     [no_sat_escape_count, no_sat_no_escape_count]]

# Perform chi-squared test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# Print the p-value
print(f"Chi-squared test p-value: {p_value}")

import statsmodels.api as sm

# Combine the 'is_estimated_escape' data and create group labels
all_escape_data = np.concatenate([sat_escape, no_sat_escape])
group_labels = np.array(["sat"] * len(sat_escape) + ["no_sat"] * len(no_sat_escape))

# Convert group labels to numerical values: 1 for 'sat' and 0 for 'no_sat'
group_numeric = np.where(group_labels == "sat", 1, 0)

# Add a constant term for the intercept
X = sm.add_constant(group_numeric)

# Fit the binomial GLM
glm_binom = sm.GLM(all_escape_data, X, family=sm.families.Binomial())
res = glm_binom.fit()

# Print the summary
print(res.summary())


In [ ]:
# Initialize a dictionary to hold escape probabilities for each stimulus type
escape_probabilities = {}

for stimulus_type, trial_list in trials.items():
    is_estimated_escape_values = []
    for trial in trial_list:
        if len(trial['event_features']) == 0 or np.isnan(trial['event_features'].mean()):
            continue
        
        event_clusters = trial['event_clusters']
        
        if len(event_clusters) == 0 or np.isnan(event_clusters.mean()):
            continue
        
        within_frame_range = (trial['valid_event_times'][event_clusters == 0] >= stim_window[0]) & (trial['valid_event_times'][event_clusters == 0] <= stim_window[1])
        
        trial['is_estimated_escape'] = within_frame_range.sum() > 0
        is_estimated_escape_values.append(trial['is_estimated_escape'])
        
    escape_probabilities[stimulus_type] = np.mean(is_estimated_escape_values)

# Prepare data for heatmap
visual_intensities = []
auditory_intensities = []
probabilities = []

for stimulus_type, probability in escape_probabilities.items():
    visual_intensity, auditory_intensity = map(float, stimulus_type.split('_'))
    visual_intensities.append(visual_intensity)
    auditory_intensities.append(auditory_intensity)
    probabilities.append(probability)

import matplotlib.pyplot as plt
import seaborn as sns

# Create a DataFrame for easier plotting
import pandas as pd
df = pd.DataFrame({
    'Visual': visual_intensities,
    'Auditory': auditory_intensities,
    'Probability': probabilities
})

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df.pivot("Visual", "Auditory", "Probability"), cmap="Oranges", annot=True, cbar_kws={'label': 'Escape Probability'}, vmin=0, vmax=1)
plt.gca().invert_yaxis()
plt.title("Escape Probability Heatmap")
plt.savefig(f"04b - Observed probability matrix only for C-start cluster events.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# import matplotlib.pyplot as plt

# # Initialize a dictionary to store x, y coordinates grouped by category
# coords_by_category = {
#     'Control': ([], []),
#     'Visual': ([], []),
#     'Auditory': ([], []),
#     'Multisensory': ([], [])
# }

# colors = {'Control': "gray", 'Visual': "red", 'Auditory': "blue", 'Multisensory': "magenta"}

# # Loop through each stimulus type in trials
# for stim_type, trial_list in trials.items():
#     # Determine the category for this stimulus type
#     visual, auditory = map(float, stim_type.split('_'))
    
#     if visual == 0.0 and auditory == 0.0:
#         category = 'Control'
#     elif visual != 0.0 and auditory == 0.0:
#         category = 'Visual'
#     elif visual == 0.0 and auditory != 0.0:
#         category = 'Auditory'
#     else:
#         category = 'Multisensory'

#     # Initialize lists to store x, y coordinates for this category
#     x_coords, y_coords = coords_by_category[category]

#     # Loop through each trial in the list of trials for this stimulus type
#     for trial in trial_list:
# #         # Check if this trial has the 'estimated_escape' and 'is_estimated_escape' keys
# #         if 'is_estimated_escape' in trial:
# #             # If this trial is labeled as positive for estimated escapes
# #             if trial['is_estimated_escape']:
#                 # Get the estimated escapes
#                 #estimated_escapes = trial['estimated_escape']
#                 # Get the valid event times
#         if "low_D_representation" not in trial:
#             continue
        
#         valid_event_times = trial['valid_event_times']
#         # Get the low_D_representation
#         low_D_rep = trial['low_D_representation']

#         # Loop through each valid event to check the conditions
#         for i, (event_time, low_D) in enumerate(zip(valid_event_times, low_D_rep)):
#             # Check if the event is an estimated escape and falls within the relevant time window
#             #if is_escape and (stim_window[0] <= event_time <= stim_window[1]):
#             if stim_window[0] <= event_time <= stim_window[1]:
#                 # Append the x and y coordinates to the lists
#                 x_coords.append(low_D[0])
#                 y_coords.append(low_D[1])

# # Create the scatter plot
# plt.figure(figsize=(10, 8))

# # Loop through each category to plot its events
# for category, (x, y) in coords_by_category.items():
#     if category == "Multisensory" or category == "Control":
#         continue
    
#     jitter_amount = 0.02  # You can adjust this value as needed
#     x_jittered = np.array(x) + np.random.uniform(-jitter_amount, jitter_amount, len(x))
#     y_jittered = np.array(y) + np.random.uniform(-jitter_amount, jitter_amount, len(y))
#     plt.scatter(x_jittered, y_jittered, label=category, color=colors[category], s=100, alpha=0.6)

# plt.xlabel('UMAP Dimension 1')
# plt.ylabel('UMAP Dimension 2')
# plt.title('Scatter plot of estimated escapes in relevant time window')
# plt.legend(title='Category')
# plt.savefig(f"05 - 2D representation of escape events by stimulus type.pdf", format='pdf', dpi=300, bbox_inches='tight')
# plt.show()


In [ ]:
# Define custom colors based on visual intensity
custom_colors = {
    "0.0_0.0": (0.3, 0.3, 0.3),
    "0.0_1.0": (0., 0.2        , 1.        ),
    "0.3_0.0": (1.        , 0.72156863, 0.16862745),
    "0.0_0.3": (0.        , 0.83529412, 1.        ),
    "1.0_0.0": (1., 0., 0.),
    
    "0.3_0.3": (0.83529412, 0.56862745, 1),
    "0.3_1.0": (0.57254902, 0.01176471, 1),
    "1.0_0.3": (1.        , 0.56862745, 0.75686275),
    "1.0_1.0": (1.        , 0.        , 0.81568627)
}

custom_labels = {
    "0.0_0.0": "Control",
    "0.0_1.0": "Aud Max",
    "0.3_0.0": "Vis Min",
    "0.0_0.3": "Aud Min",
    "1.0_0.0": "Vis Max",
    
    "0.3_0.3": "Vis Min + Aud Min",
    "0.3_1.0": "Vis Min + Aud Max",
    "1.0_0.3": "Vis Max + Aud Min",
    "1.0_1.0": "Vis Max + Aud Max"
}

In [ ]:
#os.chdir(r"C:\Users\PC\Desktop\Resultados")

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patheffects import withStroke

# Initialize a dictionary to store x, y coordinates grouped by category
coords_by_category = {}
colors = {}

# Loop through each stimulus type in trials
for stim_type, trial_list in trials.items():
    # Determine the category for this stimulus type
    visual, auditory = map(float, stim_type.split('_'))
    
    if visual == 0.0 and auditory == 0.0:
        category = stim_type
        if category not in coords_by_category:
            coords_by_category[category] = ([], [])
            colors[category] = custom_colors[stim_type]
    elif visual != 0.0 and auditory == 0.0:
        category = stim_type
        if category not in coords_by_category:
            coords_by_category[category] = ([], [])
            colors[category] = custom_colors[stim_type]
    elif visual == 0.0 and auditory != 0.0:
        category = stim_type
        if category not in coords_by_category:
            coords_by_category[category] = ([], [])
            colors[category] = custom_colors[stim_type]

    # Skip multisensory
    else:
        continue

    # Initialize lists to store x, y coordinates for this category
    x_coords, y_coords = coords_by_category[category]

    for trial in trial_list:
        if "low_D_representation" not in trial:
            continue
        
        valid_event_times = trial['valid_event_times']
        low_D_rep = trial['low_D_representation']

        for i, (event_time, low_D) in enumerate(zip(valid_event_times, low_D_rep)):
            if stim_window[0] <= event_time <= stim_window[1]:
                x_coords.append(low_D[0])
                y_coords.append(low_D[1])

# Create the scatter plot
plt.figure(figsize=(10, 8))

unisensory_centroids = {}

# Plot centroids and events
for category, (x, y) in coords_by_category.items():
    jitter_amount = 0.0
    x_jittered = np.array(x) + np.random.uniform(-jitter_amount, jitter_amount, len(x))
    y_jittered = np.array(y) + np.random.uniform(-jitter_amount, jitter_amount, len(y))
    plt.scatter(x_jittered, y_jittered, label=custom_labels[category], color=colors[category], s=130, alpha=0.8,edgecolor='black')
    
    if category != "0.0_0.0":
        centroid_x = np.median(x)
        centroid_y = np.median(y)
        unisensory_centroids[category] = (centroid_x, centroid_y)
        
        plt.text(centroid_x, centroid_y, 'x', color=colors[category], fontsize=30, ha='center', va='center',path_effects=[withStroke(linewidth=5, foreground='black')])

plt.xlabel('Dimension 1', fontsize=16)
plt.ylabel('Dimension 2', fontsize=16)
plt.title('Scatter plot of estimated escapes in relevant time window', fontsize=18)
plt.legend(fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlim(-0.55,0.8)
plt.ylim(-0.5,0.6)
plt.savefig(f"05 - 2D representation of escape events by stimulus type.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import seaborn as sns

# Initialize the plot
plt.figure(figsize=(10, 6))

# Loop over each category to plot its density
for category, (x, y) in coords_by_category.items():
    if custom_labels[category] == "Control":
        continue
    sns.kdeplot(x, bw_adjust=1.0, label=custom_labels[category], color=custom_colors[category], fill=True, edgecolor='black')

plt.xlabel('Dimension 1', fontsize=16)
plt.title('Density plot along Dimension 1', fontsize=18)
plt.legend(fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlim(-0.55, 0.8)

# Save the plot
plt.savefig(f"06 - Density plot along Dimension 1.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

# Initialize the plot
plt.figure(figsize=(10, 6))

# Loop over each category to plot its density
for category, (x, y) in coords_by_category.items():
    if custom_labels[category] == "Control":
        continue
    sns.kdeplot(y, bw_adjust=1.0, label=custom_labels[category], color=custom_colors[category], fill=True, edgecolor='black')

plt.xlabel('Dimension 2', fontsize=16)
plt.title('Density plot along Dimension 2', fontsize=18)
plt.legend(fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlim(-0.5, 0.6)

# Save the plot
plt.savefig(f"06 - Density plot along Dimension 2.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Initialize a dictionary to store x, y coordinates grouped by category
coords_by_category = {}

# Loop through each stimulus type in trials
for stim_type, trial_list in trials.items():
    # Determine the category for this stimulus type
    visual, auditory = map(float, stim_type.split('_'))

    # Focus only on multisensory stimuli
    if visual != 0.0 and auditory != 0.0:
        category = stim_type
        coords_by_category[category] = ([], [])

    else:
        continue

    # Initialize lists to store x, y coordinates for this category
    x_coords, y_coords = coords_by_category[category]

    for trial in trial_list:
        if "low_D_representation" not in trial:
            continue

        valid_event_times = trial['valid_event_times']
        low_D_rep = trial['low_D_representation']

        for i, (event_time, low_D) in enumerate(zip(valid_event_times, low_D_rep)):
            if stim_window[0] <= event_time <= stim_window[1]:
                x_coords.append(low_D[0])
                y_coords.append(low_D[1])

# Create the scatter plot
plt.figure(figsize=(10, 8))

# Plot centroids and events
for category, (x, y) in coords_by_category.items():
    jitter_amount = 0.02
    x_jittered = np.array(x) + np.random.uniform(-jitter_amount, jitter_amount, len(x))
    y_jittered = np.array(y) + np.random.uniform(-jitter_amount, jitter_amount, len(y))

    plt.scatter(x_jittered, y_jittered, label=custom_labels[category], color=custom_colors[category], s=130, alpha=0.8, edgecolor='black')
    
for category, (x, y) in unisensory_centroids.items():
    plt.text(x, y, 'x', color=colors[category], fontsize=30, ha='center', va='center',path_effects=[withStroke(linewidth=5, foreground='black')])
    

plt.xlabel('Dimension 1', fontsize=16)
plt.ylabel('Dimension 2', fontsize=16)
plt.title('Scatter plot of multisensory events by specific stimulus', fontsize=18)
plt.legend(fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlim(-0.55,0.8)
plt.ylim(-0.5,0.6)
plt.savefig(f"07 - Multisensory 2D representation by specific stimulus.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import seaborn as sns

# Initialize the plot
plt.figure(figsize=(10, 6))

# Loop over each category to plot its density
for category, (x, y) in coords_by_category.items():
    if custom_labels[category] == "Control":
        continue
    sns.kdeplot(x, bw_adjust=1.0, label=custom_labels[category], color=custom_colors[category], fill=True, edgecolor='black')

plt.xlabel('Dimension 1', fontsize=16)
plt.title('Density plot along Dimension 1', fontsize=18)
plt.legend(fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlim(-0.55, 0.8)

# Save the plot
plt.savefig(f"07 - Density plot along Dimension 1.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

# Initialize the plot
plt.figure(figsize=(10, 6))

# Loop over each category to plot its density
for category, (x, y) in coords_by_category.items():
    if custom_labels[category] == "Control":
        continue
    sns.kdeplot(y, bw_adjust=1.0, label=custom_labels[category], color=custom_colors[category], fill=True, edgecolor='black')

plt.xlabel('Dimension 2', fontsize=16)
plt.title('Density plot along Dimension 2', fontsize=18)
plt.legend(fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlim(-0.5, 0.6)

# Save the plot
plt.savefig(f"07 - Density plot along Dimension 2.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
np.array([255, 0, 208])/255

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Initialize a dictionary to store x, y coordinates specifically for multisensory stimuli
# coords_by_multisensory_stim = {}

# # # Define custom colors based on visual intensity
# # custom_colors = {
# #     "0.3_0.3": (0.83529412, 0.56862745, 1),
# #     "0.3_1.0": (0.57254902, 0.01176471, 1),
# #     "1.0_0.3": (1.        , 0.56862745, 0.75686275),
# #     "1.0_1.0": (0.98039216, 0.00784314, 0.38431373)
# # }

# # Loop through each stimulus type in trials
# for stim_type, trial_list in trials.items():
#     visual, auditory = map(float, stim_type.split('_'))

#     # Only populate for multisensory stimuli
#     if visual != 0.0 and auditory != 0.0:
#         if stim_type not in coords_by_multisensory_stim:
#             coords_by_multisensory_stim[stim_type] = ([], [])
#         x_coords, y_coords = coords_by_multisensory_stim[stim_type]

#         # Loop through each trial in the list of trials for this stimulus type
#         for trial in trial_list:
#             if 'low_D_representation' in trial:
#                 #estimated_escapes = trial['estimated_escape']
#                 valid_event_times = trial['valid_event_times']
#                 low_D_rep = trial['low_D_representation']

#                 for i, (event_time, low_D) in enumerate(zip(valid_event_times, low_D_rep)):
#                     if stim_window[0] <= event_time <= stim_window[1]:
#                         x_coords.append(low_D[0])
#                         y_coords.append(low_D[1])

# # Create the scatter plot
# plt.figure(figsize=(10, 8))

# # Loop through each multisensory type to plot its events
# for stim_type, (x, y) in coords_by_multisensory_stim.items():
#     plt.scatter(x, y, label=stim_type, color=custom_colors[stim_type], s=100, alpha=0.8)

# plt.xlabel('UMAP Dimension 1')
# plt.ylabel('UMAP Dimension 2')
# plt.title('Scatter plot of estimated escapes in relevant time window for multisensory stimuli')
# plt.legend(title='Category')
# plt.savefig(f"06 - 2D representation of multisensory escape events by stimulus type.pdf", format='pdf', dpi=300, bbox_inches='tight')
# plt.show()



In [ ]:
# Initialize a dictionary to store counts
cluster_counts = {}

# Define custom colors based on visual intensity and add colors for unisensory and control
custom_colors = {
    "0.3_0.3": (0.83529412, 0.56862745, 1),
    "0.3_1.0": (0.57254902, 0.01176471, 1),
    "1.0_0.3": (1.        , 0.56862745, 0.75686275),
    "1.0_1.0": (0.98039216, 0.00784314, 0.38431373),
    "0.0_0.0": "gray",
    "0.3_0.0": (1.        , 0.61568627, 0.45098039),
    "1.0_0.0": (1.        , 0.30196078, 0.        ),
    "0.0_0.3": (0.49019608, 0.70980392, 1.        ),
    "0.0_1.0": (0.        , 0.43137255, 1.        )
}

# Loop through each stimulus type in trials
for stim_type, trial_list in trials.items():
    total_events = 0
    cluster_0_events = 0
    
    for trial in trial_list:
        if 'is_estimated_escape' in trial and trial['is_estimated_escape']:
            valid_event_times = trial['valid_event_times']
            event_clusters = trial['event_clusters']
            
            for event_time, cluster in zip(valid_event_times, event_clusters):
                if stim_window[0] <= event_time <= stim_window[1]:
                    total_events += 1
                    if cluster == 0:
                        cluster_0_events += 1
    
    # Store the counts for this stimulus type
    cluster_counts[stim_type] = (cluster_0_events, total_events)

# Calculate proportions
proportions = {}
for stim_type, (cluster_0, total) in cluster_counts.items():
    if total > 0:
        proportions[stim_type] = cluster_0 / total
    else:
        proportions[stim_type] = 0

# Define a custom order for the stimulus types
custom_order = ["0.0_0.0", "0.3_0.0", "1.0_0.0", "0.0_0.3", "0.0_1.0", "0.3_0.3", "0.3_1.0", "1.0_0.3", "1.0_1.0"]

# Filter out any stim types not in our dataset (this is optional but good for robustness)
custom_order = [stim for stim in custom_order if stim in proportions]

# Get the proportion values and colors in the custom order
proportion_values_ordered = [proportions[stim] for stim in custom_order]
colors_ordered = [custom_colors[stim] for stim in custom_order]

# Plot the ordered proportions
fig, ax = plt.subplots(figsize=(12, 8))

ax.bar(custom_order, proportion_values_ordered, color=colors_ordered)
ax.set_xlabel('Stimulus Type')
ax.set_ylabel('Proportion in C-Start Cluster')
ax.set_title('Proportion of Events in C-Start Cluster by Stimulus Type')
plt.savefig(f"07 - Proportion of events in C-start cluster by stimulus type.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import numpy as np
import pandas as pd

# Initialize a dictionary to store centroids for each stimulus type
centroids = {}

for stim_type, trials_i in trials.items():
    # Initialize a list to store low_D_representation for this stimulus type
    low_D_points = []
    
    for trial in trials_i:
        if 'is_estimated_escape' in trial and trial['is_estimated_escape']:
            valid_indices = np.where((trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1]))[0]
            if len(valid_indices) > 0:
                low_D_points.extend(trial['low_D_representation'][valid_indices])
    
    # Convert to numpy array for easier calculations
    low_D_points = np.array(low_D_points)
    
    # Calculate centroid for this stimulus type
    centroid = np.median(low_D_points, axis=0)
    centroids[stim_type] = centroid

In [ ]:
distances_dict = {}

for stim_type, trials_i in trials.items():
    visual_intensity, auditory_intensity = map(float, stim_type.split('_'))
    
    # Skip unisensory and null stimuli
    if visual_intensity == 0.0 or auditory_intensity == 0.0:
        continue
    
    # Initialize a list to store low_D_representation for this stimulus type
    low_D_points = []
    
    for trial in trials_i:
        if 'is_estimated_escape' in trial and trial['is_estimated_escape']:
            valid_indices = np.where((trial['valid_event_times'] >= stim_window[0]) & (trial['valid_event_times'] <= stim_window[1]))[0]
            if len(valid_indices) > 0:
                low_D_points.extend(trial['low_D_representation'][valid_indices])
    
    # Convert to numpy array for easier calculations
    low_D_points = np.array(low_D_points)
    
    visual_centroid = centroids[f"{visual_intensity}_0.0"]
    auditory_centroid = centroids[f"0.0_{auditory_intensity}"]
    
    distances = []
    
    for low_D_rep in low_D_points:
        # Calculate distances to the visual and auditory centroids
        distance_to_visual = np.linalg.norm(low_D_rep - visual_centroid)
        distance_to_auditory = np.linalg.norm(low_D_rep - auditory_centroid)

        distances.append([distance_to_visual, distance_to_auditory])
    
    # Convert distances to a numpy array for easier manipulation
    distances = np.array(distances)

    # Calculate the standard deviations for visual and auditory distances
#     std_visual = np.std(distances[:, 0])
#     std_auditory = np.std(distances[:, 1])

#     # Normalize the distances
#     distances[:, 0] /= std_visual
#     distances[:, 1] /= std_auditory

    # Add the normalized distances to your distances_dict
    distances_dict[stim_type] = distances

In [ ]:
# Initialize the scatterplot
fig, ax = plt.subplots(figsize=(8,8))

# Loop through each multisensory stimulus type and plot
for stim_type, distances in distances_dict.items():
    ax.scatter(distances[:, 0], distances[:, 1], label=custom_labels[stim_type], color=custom_colors[stim_type], s=130, alpha=0.8, edgecolor='black')

# Add labels and title
ax.set_xlabel('Distance to Visual Centroid',fontsize=14)
ax.set_ylabel('Distance to Auditory Centroid',fontsize=14)
#ax.set_title('Distances to Visual and Auditory Centroids for Multisensory Stimuli')
plt.legend(fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)

# Get the limits for both axes
x_lim = ax.get_xlim()
y_lim = ax.get_ylim()

# Calculate the common limits for the identity line
common_lim = [min(x_lim[0], y_lim[0]), max(x_lim[1], y_lim[1])]

# Plot the identity line
ax.plot(common_lim, common_lim, '--', linewidth=2, color='gray', label='Identity Line')

# Optionally, set the axes limits back to the common limits
ax.set_xlim(common_lim)
ax.set_ylim(common_lim)

# Show the plot
plt.savefig(f"08 - Distance from multisensory events to unisensory centroids.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Initialize the plot
plt.figure(figsize=(10, 6))

# Loop through each multisensory stimulus type and plot its density
for stim_type, distances in distances_dict.items():
    sns.kdeplot(distances[:, 0], bw_adjust=1.0, label=custom_labels[stim_type], color=custom_colors[stim_type], fill=True, edgecolor='black')

plt.xlabel('Distance to Visual Centroid', fontsize=16)
plt.title('Density plot for Distance to Visual Centroid', fontsize=18)
plt.legend(fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlim(common_lim)  # Assuming common_lim is defined in your workspace

# Save the plot
plt.savefig(f"08 - Density plot for Distance to Visual Centroid.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

# Initialize the plot
plt.figure(figsize=(10, 6))

# Loop through each multisensory stimulus type and plot its density
for stim_type, distances in distances_dict.items():
    sns.kdeplot(distances[:, 1], bw_adjust=1.0, label=custom_labels[stim_type], color=custom_colors[stim_type], fill=True, edgecolor='black')

plt.xlabel('Distance to Auditory Centroid', fontsize=16)
plt.title('Density plot for Distance to Auditory Centroid', fontsize=18)
plt.legend(fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlim(common_lim)  # Assuming common_lim is defined in your workspace

# Save the plot
plt.savefig(f"08 - Density plot for Distance to Auditory Centroid.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['Stimulus_Type', 'Component', 'Distance'])

# Loop through the distances_dict to populate the DataFrame
for stim_type, distances in distances_dict.items():
    for distance in distances:
        df = df.append({'Stimulus_Type': stim_type, 'Component': 'Visual', 'Distance': distance[0]}, ignore_index=True)
        df = df.append({'Stimulus_Type': stim_type, 'Component': 'Auditory', 'Distance': distance[1]}, ignore_index=True)

df['Stimulus_Type'] = pd.Categorical(df['Stimulus_Type'], categories=sorted(distances_dict.keys()), ordered=True)
df.sort_values('Stimulus_Type', inplace=True)

# Set up the matplotlib figure
fig, axs = plt.subplots(1, 2, figsize=(15, 7))

# Create boxplot + swarmplot for Visual Component
sns.boxplot(x='Stimulus_Type', y='Distance', data=df[df['Component'] == 'Visual'], ax=axs[0], palette=custom_colors, hue_order=sorted(distances_dict.keys()))
sns.swarmplot(x='Stimulus_Type', y='Distance', data=df[df['Component'] == 'Visual'], ax=axs[0], color='black')
axs[0].set_title('Visual Component', fontsize=14)
axs[0].set_ylabel('Distance to Visual Centroid', fontsize=14)
axs[0].set_xlabel('')
axs[0].tick_params(axis='both', which='major', labelsize=14)
axs[0].set_xticklabels([custom_labels[stim_type] for stim_type in sorted(distances_dict.keys())], rotation=10)

# Create boxplot + swarmplot for Auditory Component
sns.boxplot(x='Stimulus_Type', y='Distance', data=df[df['Component'] == 'Auditory'], ax=axs[1], palette=custom_colors, hue_order=sorted(distances_dict.keys()))
sns.swarmplot(x='Stimulus_Type', y='Distance', data=df[df['Component'] == 'Auditory'], ax=axs[1], color='black')
axs[1].set_title('Auditory Component', fontsize=14)
axs[1].set_ylabel('Distance to Auditory Centroid', fontsize=14)
axs[1].tick_params(axis='both', which='major', labelsize=14)
axs[1].set_xlabel('')
axs[1].set_xticklabels([custom_labels[stim_type] for stim_type in sorted(distances_dict.keys())], rotation=10)
# Show the plot
plt.savefig(f"09 - Distance from multisensory events to unisensory centroids as difference in distributions.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['Stimulus_Type', 'Component', 'Distance'])

used_keys = []
# Loop through the distances_dict to populate the DataFrame
for stim_type, distances in distances_dict.items():
    for distance in distances:
        if "Aud Min" not in custom_labels[stim_type]:
            used_keys.append(stim_type)
            df = df.append({'Stimulus_Type': stim_type, 'Component': 'Visual', 'Distance': distance[0]}, ignore_index=True)
            df = df.append({'Stimulus_Type': stim_type, 'Component': 'Auditory', 'Distance': distance[1]}, ignore_index=True)

df['Stimulus_Type'] = pd.Categorical(df['Stimulus_Type'], categories=sorted(distances_dict.keys()), ordered=True)
df['Stimulus_Type'].cat.remove_unused_categories(inplace=True)
df.sort_values('Stimulus_Type', inplace=True)

used_keys = np.unique(used_keys)

# Set up the matplotlib figure
fig, axs = plt.subplots(1, 2, figsize=(15, 7))

# Create boxplot + swarmplot for Visual Component
sns.boxplot(x='Stimulus_Type', y='Distance', data=df[df['Component'] == 'Visual'], ax=axs[0], palette=custom_colors, hue_order=sorted(used_keys))
sns.swarmplot(x='Stimulus_Type', y='Distance', data=df[df['Component'] == 'Visual'], ax=axs[0], color='black')
axs[0].set_title('Visual Component', fontsize=16)
axs[0].set_ylabel('Distance to Visual Centroid', fontsize=16)
axs[0].set_xlabel('')
axs[0].tick_params(axis='both', which='major', labelsize=16)
#axs[0].set_xticks(range(len(sorted(distances_dict.keys()))))
#axs[0].set_xticklabels([custom_labels[stim_type] for stim_type in sorted(distances_dict.keys())], rotation=10)


# Create boxplot + swarmplot for Auditory Component
sns.boxplot(x='Stimulus_Type', y='Distance', data=df[df['Component'] == 'Auditory'], ax=axs[1], palette=custom_colors, hue_order=sorted(used_keys))
sns.swarmplot(x='Stimulus_Type', y='Distance', data=df[df['Component'] == 'Auditory'], ax=axs[1], color='black')
axs[1].set_title('Auditory Component', fontsize=16)
axs[1].set_ylabel('Distance to Auditory Centroid', fontsize=16)
axs[1].tick_params(axis='both', which='major', labelsize=16)
axs[1].set_xlabel('')
#axs[1].set_xticklabels([custom_labels[stim_type] for stim_type in sorted(distances_dict.keys())], rotation=10)


axs[0].set_xticks(range(len(sorted(used_keys))))
axs[0].set_xticklabels([custom_labels[stim_type] for stim_type in sorted(used_keys)], rotation=0)

axs[1].set_xticks(range(len(sorted(used_keys))))
axs[1].set_xticklabels([custom_labels[stim_type] for stim_type in sorted(used_keys)], rotation=0)


# Show the plot
plt.savefig(f"09b - Distance from multisensory events to unisensory centroids as difference in distributions.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
from scipy.stats import mannwhitneyu

# Initialize a dictionary to store the p-values for each component
p_values = {'Visual': {}, 'Auditory': {}}

components = ['Visual', 'Auditory']

for component in components:
    stimulus_types = df[df['Component'] == component]['Stimulus_Type'].unique()
    
    for i, stim_type1 in enumerate(stimulus_types):
        for j, stim_type2 in enumerate(stimulus_types):
            if i < j:  # To avoid duplicate and self-comparisons
                sample1 = df[(df['Component'] == component) & (df['Stimulus_Type'] == stim_type1)]['Distance']
                sample2 = df[(df['Component'] == component) & (df['Stimulus_Type'] == stim_type2)]['Distance']
                
                # Mann-Whitney U test
                _, p_value = mannwhitneyu(sample1, sample2)
                
                # Store the p-value
                comparison = f"{stim_type1} vs {stim_type2}"
                p_values[component][comparison] = p_value

# Number of comparisons
num_comparisons = len(p_values['Visual'])  # This will be the same for 'Visual' and 'Auditory'

# Bonferroni correction
corrected_p_values = {component: {comparison: p_value * num_comparisons for comparison, p_value in comparisons.items()} 
                      for component, comparisons in p_values.items()}

import pandas as pd

# Convert the corrected p-values to DataFrames for easy viewing
df_visual = pd.DataFrame(list(corrected_p_values['Visual'].items()), columns=['Comparison', 'Corrected_P_Value'])
df_auditory = pd.DataFrame(list(corrected_p_values['Auditory'].items()), columns=['Comparison', 'Corrected_P_Value'])

alpha = 0.05  # Significance level

# Add 'Significance' column to the Visual DataFrame
df_visual['Significance'] = df_visual['Corrected_P_Value'].apply(lambda x: '*' if x <= alpha else '')

# Add 'Significance' column to the Auditory DataFrame
df_auditory['Significance'] = df_auditory['Corrected_P_Value'].apply(lambda x: '*' if x <= alpha else '')


# Display the tables
print("Visual Component Comparisons:")
print(df_visual)
print("\nAuditory Component Comparisons:")
print(df_auditory)


In [ ]:
# from scipy.spatial.distance import euclidean

# # Initialize a 2D dictionary to store distances
# distances_matrix = {}

# for row_stim, row_trials in trials.items():
#     distances_matrix[row_stim] = {}
#     for col_stim, col_centroid in centroids.items():
#         distances_matrix[row_stim][col_stim] = []
        
#         for trial in row_trials:
#             if 'estimated_escape' in trial and 'is_estimated_escape' in trial and trial['is_estimated_escape']:
#                 valid_indices = np.where((trial['valid_event_times'] >= 10000) & (trial['valid_event_times'] <= 11300))[0]
#                 if len(valid_indices) > 0:
#                     for point in trial['low_D_representation'][valid_indices]:
#                         distance = euclidean(point, col_centroid)
#                         distances_matrix[row_stim][col_stim].append(distance)


In [ ]:
# # Order rows and columns
# ordered_stim_types = ['0.0_0.0', '0.3_0.0', '1.0_0.0', '0.0_0.3', '0.0_1.0', '0.3_0.3', '1.0_0.3', '0.3_1.0', '1.0_1.0']

# # Create an empty numpy array to store mean distances
# num_stim_types = len(ordered_stim_types)
# mean_distances_array = np.zeros((num_stim_types, num_stim_types))

# # Fill in the numpy array with mean distances
# for i, row_stim in enumerate(ordered_stim_types):
#     for j, col_stim in enumerate(ordered_stim_types):
#         distance_list = distances_matrix[row_stim][col_stim]
#         if len(distance_list) > 0:
#             mean_distance = np.mean(distance_list)
#         else:
#             mean_distance = np.nan  # Or any value to represent empty cells
#         mean_distances_array[i, j] = mean_distance

# # Plot heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(mean_distances_array, annot=True, fmt=".2f", cmap='coolwarm', 
#             xticklabels=ordered_stim_types, yticklabels=ordered_stim_types)
# plt.show()


In [ ]:
# # Create an empty numpy array to store the counts
# count_distances_array = np.zeros((num_stim_types, num_stim_types), dtype=int)

# # Fill in the numpy array with the number of distances
# for i, row_stim in enumerate(ordered_stim_types):
#     for j, col_stim in enumerate(ordered_stim_types):
#         distance_list = distances_matrix[row_stim][col_stim]
#         count_distances_array[i, j] = len(distance_list)

# # Plot heatmap for the counts
# plt.figure(figsize=(10, 8))
# sns.heatmap(count_distances_array, annot=True, fmt="d", cmap='coolwarm', 
#             xticklabels=ordered_stim_types, yticklabels=ordered_stim_types)
# plt.show()


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # Initialize a dictionary to store the centroids for each stimulus type
# centroids = {}

# # Loop through each stimulus type
# for stim_type, trials in trials.items():
#     low_D_reps = []
    
#     # Loop through each trial
#     for trial in trials:
#         # Check if the trial is estimated as an escape
#         if 'estimated_escape' in trial and trial['is_estimated_escape']:
#             # Loop through each valid event time
#             for i, event_time in enumerate(trial['valid_event_times']):
#                 # Check if the event time falls within the stimulation window
#                 if 10000 <= event_time <= 11300:
#                     # Check if this event is estimated to be an escape
#                     if trial['estimated_escape'][i]:
#                         low_D_reps.append(trial['low_D_representation'][i])
                        
#     # Calculate the centroid for this stimulus type by taking the median
#     centroid = np.median(low_D_reps, axis=0)
#     centroids[stim_type] = centroid


In [ ]:
# import numpy as np
# from collections import defaultdict

# # Initialize a dictionary to collect low-D representations for each stimulus type
# lowD_per_stimulus = defaultdict(list)

# # Iterate through all the stimulus types and their corresponding trials
# for stimulus_type, trial_list in trials.items():
#     for trial in trial_list:
#         # Check if the trial is labeled as an estimated escape
#         if 'estimated_escape' in trial and trial['is_estimated_escape']:
#             # Get valid event times
#             valid_event_times = trial['valid_event_times']
#             # Filter events that fall within the time window
#             valid_indices = np.where((valid_event_times >= 10000) & (valid_event_times <= 11300))[0]
#             # Get low_D_representation for those events
#             relevant_lowD = trial['low_D_representation'][valid_indices]
#             # Append these to the list corresponding to this stimulus type
#             lowD_per_stimulus[stimulus_type].extend(relevant_lowD)

# # Calculate the centroid for each stimulus type
# centroids = {}
# for stimulus_type, lowD_list in lowD_per_stimulus.items():
#     if len(lowD_list) > 0:
#         centroids[stimulus_type] = np.median(lowD_list, axis=0)


In [ ]:
# # Initialize a dictionary to collect distance arrays for each multisensory stimulus
# distances_per_multisensory = {}

# # Iterate through all the stimulus types and their corresponding centroids
# for stimulus_type, centroid in centroids.items():
#     visual_intensity, auditory_intensity = map(float, stimulus_type.split('_'))
#     # Check if it is a multisensory stimulus
#     if visual_intensity > 0.0 and auditory_intensity > 0.0:
#         # Initialize a list to collect distances
#         distances_list = []
#         # Iterate through the trials for this stimulus type
#         for trial in trials[stimulus_type]:
#             if 'low_D_representation' in trial and 'estimated_escape' in trial:
#                 # Filter for events that are estimated escapes
#                 estimated_escapes = trial['estimated_escape']
#                 # Get valid event times and filter for relevant window
#                 valid_event_times = trial['valid_event_times']
                
#                 print(estimated_escapes.shape, valid_event_times.shape)
                
#                 valid_indices = np.where((valid_event_times >= 10000) & (valid_event_times <= 11300) & estimated_escapes)[0]
#                 # Continue only if valid_indices is not empty
#                 if len(valid_indices) > 0:
#                     # Get the low_D_representation for these filtered events
#                     lowD_filtered_events = trial['low_D_representation'][valid_indices]
#                     # Calculate the distance to the visual and auditory centroids
#                     visual_centroid = centroids[f"{visual_intensity}_0.0"]
#                     auditory_centroid = centroids[f"0.0_{auditory_intensity}"]
#                     distances_to_visual = np.linalg.norm(lowD_filtered_events - visual_centroid, axis=1)
#                     distances_to_auditory = np.linalg.norm(lowD_filtered_events - auditory_centroid, axis=1)
#                     # Combine these into a 2D array for this trial
#                     distances_trial = np.column_stack((distances_to_visual, distances_to_auditory))
#                     distances_list.append(distances_trial)
#         # Combine these into a 2D array for this multisensory stimulus
#         if distances_list:
#             distances_per_multisensory[stimulus_type] = np.vstack(distances_list)


In [ ]:
# # Define the base colors
# gray = (0.5, 0.5, 0.5)
# light_red = (1.0, 0.7, 0.7)
# dark_red = (1.0, 0.0, 0.0)
# light_blue = (0.7, 0.7, 1.0)
# dark_blue = (0.0, 0.0, 1.0)

# # Initialize the stim_colors dictionary
# stim_colors = {}

# # Populate the dictionary with colors
# stim_colors['0.0_0.0'] = gray
# stim_colors['0.3_0.0'] = light_red
# stim_colors['1.0_0.0'] = dark_red
# stim_colors['0.0_0.3'] = light_blue
# stim_colors['0.0_1.0'] = dark_blue

# # Multisensory stimuli combinations
# stim_colors['0.3_0.3'] = (0.85, 0.7, 0.85)  # Light mix of red and blue
# stim_colors['1.0_0.3'] = (1.0, 0.0, 0.3)  # More red, less blue
# stim_colors['0.3_1.0'] = (0.3, 0.0, 1.0)  # More blue, less red
# stim_colors['1.0_1.0'] = (1.0, 0.0, 1.0)  # Full red and blue


In [ ]:
# import matplotlib.pyplot as plt

# # Initialize the scatterplot
# plt.figure(figsize=(12,12))

# # Add points for each multisensory stimulus
# for stimulus_type, distances in distances_per_multisensory.items():
#     plt.scatter(distances[:, 0], distances[:, 1], label=stimulus_type, color=stim_colors[stimulus_type])

# # Add axis labels and legend
# plt.xlabel('Distance to Visual Centroid')
# plt.ylabel('Distance to Auditory Centroid')
# plt.legend()

# # Show the plot
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12,12))

# for stimulus_type, lowD_list in lowD_per_stimulus.items():
#     lowD_array = np.array(lowD_list)
#     plt.scatter(lowD_array[:, 0], lowD_array[:, 1], label=stimulus_type, color=stim_colors[stimulus_type])
    
# for stimulus_type, centroid in centroids.items():
#     plt.scatter(centroid[0], centroid[1], marker='x', s=250, color=stim_colors[stimulus_type])

# plt.xlabel('Low-D Dimension 1')
# plt.ylabel('Low-D Dimension 2')
# plt.title('Centroids in Low-D Space')
# plt.legend()
# plt.show()


In [ ]:
# plt.figure()

# for stimulus_type, lowD_list in lowD_per_stimulus.items():
#     lowD_array = np.array(lowD_list)
#     plt.scatter(lowD_array[:, 0], lowD_array[:, 1], label=stimulus_type)

# plt.xlabel('Low-D Dimension 1')
# plt.ylabel('Low-D Dimension 2')
# plt.title('Low-D Representations by Stimulus Type')
# plt.legend()
# plt.show()


In [ ]:
# import numpy as np

# all_event_features_escapes = []
# all_is_escape_escapes = []

# # Assume `trials` is your dictionary containing all the data
# for stimulus_type, trial_list in trials.items():
#     for trial in trial_list:
        
#         if len(trial['estimated_escape']) == 0 or trial['estimated_escape'].sum() == 0:
#             continue
        
#         all_event_features_escapes.append(trial['event_features'][trial['estimated_escape']])
#         all_is_escape_escapes.append(trial['is_escape'][trial['estimated_escape']])

# all_event_features_escapes = np.vstack(all_event_features_escapes)
# all_is_escape_escapes = np.concatenate(all_is_escape_escapes)

# print(all_event_features_escapes.shape, all_is_escape_escapes.shape, all_is_escape_escapes.sum())

In [ ]:
import numpy as np

# Assuming trials is your dictionary containing all the trial information.
# Loop through each stimulus type in the trials dictionary.
for stimulus_type, trial_list in trials.items():
    # Loop through each trial in the list of trials for this stimulus type.
    for trial in trial_list:
        # Initialize the 'fish_position' dictionary within each trial.
        trial['fish_position'] = {
            'mean_eye_position': [],
            'distance_to_bottom': [],
            'distance_to_left': [],
            'vector_to_head': [],
            'angle_to_bottom': [],
            'angle_to_left': []
        }
        
        # If there are no events, the lists will remain empty.
        if len(trial['valid_event_times']) == 0:
            continue
            
        # Loop through each valid event time in the trial.
        for event_time in trial['valid_event_times']:
            # Extract the tracking data for this event time.
            tracking_data = trial['tracking_data'].loc[event_time]
            
            # Calculate the mean position of the eyes.
            mean_eye_x = np.mean([tracking_data['eyeL_x'], tracking_data['eyeR_x']])
            mean_eye_y = np.mean([tracking_data['eyeL_y'], tracking_data['eyeR_y']])
            mean_eye_position = (mean_eye_x, mean_eye_y)
            
            # Calculate the distance from the mean eye position to the bottom and left of the frame.
            distance_to_bottom = 720 - mean_eye_y
            distance_to_left = mean_eye_x
            
            # Calculate the vector pointing from the vejigaP point to the fish head (mean position of the eyes).
            vector_to_head = (mean_eye_x - tracking_data['vejigaP_x'], mean_eye_y - tracking_data['vejigaP_y'])
            
            # Calculate the angles.
            angle_to_bottom = np.arctan2(vector_to_head[1], vector_to_head[0]) * 180 / np.pi
            angle_to_left = np.arctan2(vector_to_head[0], vector_to_head[1]) * 180 / np.pi
            
            # Save these calculated values into the 'fish_position' dictionary.
            trial['fish_position']['mean_eye_position'].append(mean_eye_position)
            trial['fish_position']['distance_to_bottom'].append(distance_to_bottom)
            trial['fish_position']['distance_to_left'].append(distance_to_left)
            trial['fish_position']['vector_to_head'].append(vector_to_head)
            trial['fish_position']['angle_to_bottom'].append(angle_to_bottom)
            trial['fish_position']['angle_to_left'].append(angle_to_left)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize empty lists to hold data for scatter plots and histograms
mean_eye_positions = []
stimulus_types_for_scatter = []
distances_to_bottom = []
distances_to_left = []
angles_to_bottom = []
angles_to_left = []
stimulus_types_for_angles = []

# Loop through each stimulus type in the trials dictionary to populate the above lists
for stimulus_type, trial_list in trials.items():
    for trial in trial_list:
        fish_position = trial.get('fish_position', {})
        
        mean_eye_positions.extend(fish_position.get('mean_eye_position', []))
        stimulus_types_for_scatter.extend([stimulus_type] * len(fish_position.get('mean_eye_position', [])))
        
        distances_to_bottom.extend(fish_position.get('distance_to_bottom', []))
        distances_to_left.extend(fish_position.get('distance_to_left', []))
        
        angles_to_bottom.extend(fish_position.get('angle_to_bottom', []))
        angles_to_left.extend(fish_position.get('angle_to_left', []))
        stimulus_types_for_angles.extend([stimulus_type] * len(fish_position.get('angle_to_bottom', [])))

# Convert mean_eye_positions to x and y coordinates for easier plotting
mean_eye_positions_x = [pos[0] for pos in mean_eye_positions]
mean_eye_positions_y = [pos[1] for pos in mean_eye_positions]

# Scatter plot for mean eye positions, colored by stimulus type
plt.figure(figsize=(10, 8))
sns.scatterplot(x=mean_eye_positions_x, y=mean_eye_positions_y, hue=stimulus_types_for_scatter, palette=custom_colors)
plt.title("Scatter Plot of Mean Eye Positions by Stimulus Type")
plt.xlabel("Mean Eye X Position")
plt.ylabel("Mean Eye Y Position")
plt.legend(title='Stimulus Type', labels=[custom_labels.get(stim, stim) for stim in set(stimulus_types_for_scatter)])
plt.show()

# Histograms for distances
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(distances_to_bottom, bins=30)
plt.title("Histogram of Distances to Bottom")
plt.xlabel("Distance to Bottom")
plt.ylabel("Frequency")

plt.subplot(1, 2, 2)
sns.histplot(distances_to_left, bins=30)
plt.title("Histogram of Distances to Left")
plt.xlabel("Distance to Left")
plt.ylabel("Frequency")
plt.show()

# Histograms for angles
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(angles_to_bottom, bins=30)
plt.title("Histogram of Angles to Bottom")
plt.xlabel("Angle to Bottom")
plt.ylabel("Frequency")

plt.subplot(1, 2, 2)
sns.histplot(angles_to_left, bins=30)
plt.title("Histogram of Angles to Left")
plt.xlabel("Angle to Left")
plt.ylabel("Frequency")
plt.show()

# Scatter plot for angles, colored by stimulus type
plt.figure(figsize=(10, 8))
sns.scatterplot(x=angles_to_bottom, y=angles_to_left, hue=stimulus_types_for_angles, palette=custom_colors)
plt.title("Scatter Plot of Angles by Stimulus Type")
plt.xlabel("Angle to Bottom")
plt.ylabel("Angle to Left")
plt.legend(title='Stimulus Type', labels=[custom_labels.get(stim, stim) for stim in set(stimulus_types_for_angles)])
plt.show()


In [ ]:
import pandas as pd
import statsmodels.api as sm

# Initialize empty lists to collect data
low_D_dimension1 = []
distances_to_bottom = []
distances_to_left = []
angles_to_bottom = []
angles_to_left = []
visual_intensities = []
auditory_intensities = []

# Populate the lists with data from all trials
for stimulus_type, trial_list in trials.items():
    visual_intensity, auditory_intensity = map(float, stimulus_type.split('_'))
    
    for trial in trial_list:
        fish_position = trial.get('fish_position', {})
        low_D_rep = trial.get('low_D_representation', [])
        
        num_events = len(fish_position.get('distance_to_bottom', []))
        if num_events > 0:
            low_D_dimension1.extend(low_D_rep[:, 0])  # Assuming low_D_rep is a numpy array
            
            distances_to_bottom.extend(fish_position['distance_to_bottom'])
            distances_to_left.extend(fish_position['distance_to_left'])
            angles_to_bottom.extend(fish_position['angle_to_bottom'])
            angles_to_left.extend(fish_position['angle_to_left'])
            
            visual_intensities.extend([visual_intensity] * num_events)
            auditory_intensities.extend([auditory_intensity] * num_events)

# Create a DataFrame
data = pd.DataFrame({
    'low_D_dimension1': low_D_dimension1,
    'distance_to_bottom': distances_to_bottom,
    'distance_to_left': distances_to_left,
    'angle_to_bottom': angles_to_bottom,
    'angle_to_left': angles_to_left,
    'visual_intensity': visual_intensities,
    'auditory_intensity': auditory_intensities
})

# Add a constant term for the intercept
data = sm.add_constant(data)

In [ ]:
# Create the GLM model
model = sm.GLM(data['low_D_dimension1'], data[['const', 'distance_to_bottom', 'distance_to_left', 'angle_to_bottom', 'angle_to_left']])
result = model.fit()

# Print the summary
print("GLM Model Summary:")
print(result.summary())


In [ ]:
# Create the GLM model
model = sm.GLM(data['low_D_dimension1'], data[['const', 'distance_to_bottom']])
result = model.fit()

# Print the summary
print("GLM Model Summary:")
print(result.summary())

In [ ]:
# Filter data for multisensory stimuli where the visual component is 1.0
filtered_data = data[(data['visual_intensity'] > 0.0) & (data['auditory_intensity'] > 0.0)]

# Create the filtered GLM model
filtered_model = sm.GLM(filtered_data['low_D_dimension1'], filtered_data[['const', 'distance_to_bottom', 'distance_to_left', 'angle_to_bottom', 'angle_to_left']])
filtered_result = filtered_model.fit()

# Print the summary of the filtered model
print("Filtered GLM Model Summary:")
print(filtered_result.summary())


In [ ]:
# Filter data for multisensory stimuli where the visual component is 1.0
filtered_data = data[(data['visual_intensity'] > 0.0) & (data['auditory_intensity'] > 0.0)]

# Create the filtered GLM model
filtered_model = sm.GLM(filtered_data['low_D_dimension1'], filtered_data[['const', 'distance_to_bottom']])
filtered_result = filtered_model.fit()

# Print the summary of the filtered model
print("Filtered GLM Model Summary:")
print(filtered_result.summary())

In [ ]:
# Scatter plots
plt.figure(figsize=(12, 12))

# Distance to bottom
plt.subplot(2, 2, 1)
plt.scatter(filtered_data['distance_to_bottom'], filtered_data['low_D_dimension1'], alpha=0.5)
plt.title('Distance to Bottom vs. Low-D Dimension 1')
plt.xlabel('Distance to Bottom')
plt.ylabel('Low-D Dimension 1')

# Distance to left
plt.subplot(2, 2, 2)
plt.scatter(filtered_data['distance_to_left'], filtered_data['low_D_dimension1'], alpha=0.5)
plt.title('Distance to Left vs. Low-D Dimension 1')
plt.xlabel('Distance to Left')
plt.ylabel('Low-D Dimension 1')

# Angle to bottom
plt.subplot(2, 2, 3)
plt.scatter(filtered_data['angle_to_bottom'], filtered_data['low_D_dimension1'], alpha=0.5)
plt.title('Angle to Bottom vs. Low-D Dimension 1')
plt.xlabel('Angle to Bottom')
plt.ylabel('Low-D Dimension 1')

# Angle to left
plt.subplot(2, 2, 4)
plt.scatter(filtered_data['angle_to_left'], filtered_data['low_D_dimension1'], alpha=0.5)
plt.title('Angle to Left vs. Low-D Dimension 1')
plt.xlabel('Angle to Left')
plt.ylabel('Low-D Dimension 1')

plt.tight_layout()
plt.show()


In [ ]:
# Filter data for multisensory stimuli where the visual component is 1.0
filtered_data = data[(data['visual_intensity'] > 0.0)]

# Create the filtered GLM model
filtered_model = sm.GLM(filtered_data['low_D_dimension1'], filtered_data[['const', 'distance_to_bottom']])
filtered_result = filtered_model.fit()

# Print the summary of the filtered model
print("Filtered GLM Model Summary:")
print(filtered_result.summary())

In [ ]:
# Filter data for multisensory stimuli where the visual component is 1.0
filtered_data = data[(data['visual_intensity'] == 0.0)]

# Create the filtered GLM model
filtered_model = sm.GLM(filtered_data['low_D_dimension1'], filtered_data[['const', 'distance_to_bottom']])
filtered_result = filtered_model.fit()

# Print the summary of the filtered model
print("Filtered GLM Model Summary:")
print(filtered_result.summary())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def calculate_heading_and_C1(trials):
    for stimulus_type, stimulus_trials in trials.items():
        heading_angles_list = []
        C1_durations = []
        C1_angles = []
        max_angular_velocities = []
        
        for trial in stimulus_trials:
            tracking_data = trial['tracking_data']
            valid_event_times = trial['valid_event_times']
            
            heading_angles_per_trial = []
            C1_durations_per_trial = []
            C1_angles_per_trial = []
            max_angular_velocities_per_trial = []
            
            for event_time in valid_event_times:
                # Calculate the initial vector at the event time
                initial_vector = np.array([
                    tracking_data.loc[event_time, 'vejigaP_x'] - tracking_data.loc[event_time, 'eye_mean_x'],
                    tracking_data.loc[event_time, 'vejigaP_y'] - tracking_data.loc[event_time, 'eye_mean_y']
                ])
                
                # Initialize list to store heading angles for this event
                heading_angles = []
                
                for i in range(30):  # For each of the first 30 frames after the event
                    current_frame = event_time + i
                    
                    # Calculate the current vector
                    current_vector = np.array([
                        tracking_data.loc[current_frame, 'vejigaP_x'] - tracking_data.loc[current_frame, 'eye_mean_x'],
                        tracking_data.loc[current_frame, 'vejigaP_y'] - tracking_data.loc[current_frame, 'eye_mean_y']
                    ])
                    
                    # Calculate the angle between the initial and current vector
                    angle = np.arccos(np.dot(initial_vector, current_vector) / 
                                      (np.linalg.norm(initial_vector) * np.linalg.norm(current_vector)))
                    
                    heading_angles.append(angle)
                
                # Save the heading angles for this event
                heading_angles_per_trial.append(np.array(heading_angles))
                
                # Calculate C1 duration and angle
                derivative = np.diff(np.abs(heading_angles))
                C1_end_frame = np.where(derivative <= 0)[0][0] - 1
                C1_durations_per_trial.append(C1_end_frame)
                C1_angles_per_trial.append(np.abs(heading_angles[C1_end_frame]))
                
                # Calculate angular velocity (change in heading angle between each frame)
                angular_velocity = np.diff(heading_angles)

                # Store the max angular velocity for this event
                max_angular_velocities_per_trial.append(np.max(np.abs(angular_velocity)))
                
            # Save all event data for this trial
            heading_angles_list.append(heading_angles_per_trial)
            C1_durations.append(C1_durations_per_trial)
            C1_angles.append(C1_angles_per_trial)
            max_angular_velocities.append(max_angular_velocities_per_trial)
            
        for i, trial in enumerate(stimulus_trials):
            trial['heading_angles'] = heading_angles_list[i]
            trial['C1_durations'] = C1_durations[i]
            trial['C1_angles'] = C1_angles[i]
            trial['max_angular_velocities'] = max_angular_velocities[i]

# Assume `trials` is your dictionary containing all the trial data
calculate_heading_and_C1(trials)


In [ ]:
import random

# Conversion factor from frames to milliseconds
frame_to_ms = 1000 / 437

def visualize_data(trials, custom_colors):
    # Plot 10 random heading angle time series
    plt.figure(figsize=(12, 8))
    
    for i in range(10):
        # Ensure we only choose trials that have at least one valid event
        while True:
            stimulus_type, stimulus_trials = random.choice(list(trials.items()))
            trial = random.choice(stimulus_trials)
            if len(trial['valid_event_times']) > 0:
                break
        event_index = random.randint(0, len(trial['valid_event_times']) - 1)
        # In the section where you plot the 10 random heading angle time series
        plt.xlabel('Time (ms)')

        # Convert frame numbers to milliseconds
        time_in_ms = np.arange(30) * frame_to_ms
        plt.plot(time_in_ms, trial['heading_angles'][event_index], label=f"Stimulus: {stimulus_type}, Event: {event_index}")

    plt.xlabel('Frame')
    plt.ylabel('Heading Angle (radians)')
    plt.title('Randomly Selected Heading Angle Time Series')
    plt.legend()
    plt.show()

    # Prepare data for histograms
    all_C1_durations = []
    all_C1_angles = []
    cluster_0_durations = []
    cluster_0_angles = []
    cluster_neg1_durations = []
    cluster_neg1_angles = []
    
    for stimulus_type, stimulus_trials in trials.items():
        for trial in stimulus_trials:
            all_C1_durations.extend(trial['C1_durations'])
            all_C1_angles.extend(trial['C1_angles'])
            
            event_clusters = trial['event_clusters']
            for i, cluster in enumerate(event_clusters):
                if cluster == 0:
                    cluster_0_durations.append(trial['C1_durations'][i])
                    cluster_0_angles.append(trial['C1_angles'][i])
                elif cluster == -1:
                    cluster_neg1_durations.append(trial['C1_durations'][i])
                    cluster_neg1_angles.append(trial['C1_angles'][i])

    # Plot histograms for all C1 durations and angles
    plt.figure(figsize=(12, 6))
    # Define common bins for C1 durations and angles
    bin_edges_durations = np.linspace(min(all_C1_durations), max(all_C1_durations), 16) * frame_to_ms
    bin_edges_angles = np.linspace(min(all_C1_angles), max(all_C1_angles), 31)

    # Convert C1 durations to milliseconds
    all_C1_durations_ms = np.array(all_C1_durations) * frame_to_ms
    cluster_0_durations_ms = np.array(cluster_0_durations) * frame_to_ms
    cluster_neg1_durations_ms = np.array(cluster_neg1_durations) * frame_to_ms

    # Use the common bins in histograms
    plt.hist(all_C1_durations_ms, bins=bin_edges_durations, color='b', alpha=0.7, label='C1 Duration')
    plt.xlabel('C1 Duration (ms)')
    plt.ylabel('Frequency')
    plt.title('Histogram of C1 Durations')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.hist(all_C1_angles, bins=30, color='r', alpha=0.7, label='C1 Angle')
    plt.xlabel('C1 Angle (radians)')
    plt.ylabel('Frequency')
    plt.title('Histogram of C1 Angles')
    plt.legend()
    plt.show()

    # Create mirrored histograms
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))

    axes[0].hist(cluster_0_durations_ms, bins=bin_edges_durations, color='g', alpha=0.7, label='Cluster 0', density=True)
    axes[0].hist(cluster_neg1_durations_ms, bins=bin_edges_durations, color='m', alpha=0.7, label='Cluster -1', density=True)
    axes[0].set_xlabel('C1 Duration (ms)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Mirrored Histogram of C1 Durations by Cluster')
    axes[0].legend()

    axes[1].hist(cluster_0_angles, bins=16, color='g', alpha=0.7, label='Cluster 0', density=True)
    axes[1].hist(cluster_neg1_angles, bins=16, color='m', alpha=0.7, label='Cluster -1', density=True)
    axes[1].set_xlabel('C1 Angle (radians)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Mirrored Histogram of C1 Angles by Cluster')
    axes[1].legend()

    plt.show()

# Assume `trials` is your dictionary containing all the trial data
# Assume `custom_colors` is your custom colors dictionary
# calculate_heading_and_C1(trials)  # Uncomment if not run previously
visualize_data(trials, custom_colors)


In [ ]:
# Conversion factors
frame_to_ms = 437 / 1000
radian_to_degree = 180 / np.pi

# Initialize lists to store max angular velocities
cluster_0_max_angular_velocities = []
cluster_neg1_max_angular_velocities = []

# Loop over trials to collect data
for stimulus_type, stimulus_trials in trials.items():
    for trial in stimulus_trials:
        max_angular_velocities = np.array(trial['max_angular_velocities'])  # in radians/frame
        event_clusters = trial['event_clusters']
        
        for i, cluster in enumerate(event_clusters):
            max_angular_velocity_deg_per_ms = max_angular_velocities[i] * radian_to_degree * frame_to_ms
            if cluster == 0:
                cluster_0_max_angular_velocities.append(max_angular_velocity_deg_per_ms)
            elif cluster == -1:
                cluster_neg1_max_angular_velocities.append(max_angular_velocity_deg_per_ms)

# Plot histograms
plt.figure(figsize=(12, 6))
plt.hist(cluster_0_max_angular_velocities, bins=30, alpha=0.7, label='Cluster 0', color='g', density=True)
plt.hist(cluster_neg1_max_angular_velocities, bins=30, alpha=0.7, label='Cluster -1', color='m', density=True)
plt.xlabel('Max Angular Velocity (degrees/ms)')
plt.ylabel('Frequency')
plt.title('Histogram of Max Angular Velocity by Cluster')
plt.legend()
plt.show()


In [ ]:
import numpy as np

def calculate_stats(trials):
    # Conversion factors
    frame_to_ms = 1000 / 437
    radian_to_degree = 180 / np.pi

    # Initialize lists to collect data for each cluster
    cluster_0_C1_angles = []
    cluster_0_C1_durations = []
    cluster_0_max_angular_velocities = []

    cluster_neg1_C1_angles = []
    cluster_neg1_C1_durations = []
    cluster_neg1_max_angular_velocities = []

    # Loop over trials to collect data
    for stimulus_type, stimulus_trials in trials.items():
        for trial in stimulus_trials:
            C1_angles = np.array(trial['C1_angles'])
            C1_durations = np.array(trial['C1_durations']) * frame_to_ms  # convert to ms
            max_angular_velocities = np.array(trial['max_angular_velocities']) * radian_to_degree / frame_to_ms  # convert to degrees/ms
            event_clusters = trial['event_clusters']
            
            for i, cluster in enumerate(event_clusters):
                if cluster == 0:
                    cluster_0_C1_angles.append(C1_angles[i] * radian_to_degree)  # convert to degrees
                    cluster_0_C1_durations.append(C1_durations[i])
                    cluster_0_max_angular_velocities.append(max_angular_velocities[i])
                elif cluster == -1:
                    cluster_neg1_C1_angles.append(C1_angles[i] * radian_to_degree)  # convert to degrees
                    cluster_neg1_C1_durations.append(C1_durations[i])
                    cluster_neg1_max_angular_velocities.append(max_angular_velocities[i])

    # Calculate and print statistics for each cluster
    for cluster, data in {'Cluster 0': [cluster_0_C1_angles, cluster_0_C1_durations, cluster_0_max_angular_velocities], 
                          'Cluster -1': [cluster_neg1_C1_angles, cluster_neg1_C1_durations, cluster_neg1_max_angular_velocities]}.items():
        print(f"Statistics for {cluster}:")
        print(f"  Mean C1 Angle: {np.nanmean(data[0]):.2f} degrees, Std Dev: {np.nanstd(data[0]):.2f} degrees")
        print(f"  Mean C1 Duration: {np.nanmean(data[1]):.2f} ms, Std Dev: {np.nanstd(data[1]):.2f} ms")
        print(f"  Mean Max Angular Velocity: {np.nanmean(data[2]):.2f} degrees/ms, Std Dev: {np.nanstd(data[2]):.2f} degrees/ms")

# Run the function
# Assume `trials` is your dictionary containing all the trial data
calculate_stats(trials)


In [ ]:
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel('G:\\My Drive\\Proyectos\\Zebrafish Multisensory Integration\\Cstart Escape Response and Integration - Behavioral Paradigm\\Análisis\\F_stim_data.xlsx')

# Loop through the DataFrame and update the 'trials' dictionary
for index, row in df.iterrows():
    trial_number = row['Trial number']
    F_stim = row['F_stim']
    
    # Loop through trials dictionary to find the matching trial number and update it
    for stimulus_type, stimulus_trials in trials.items():
        for trial in stimulus_trials:
            if trial['metadata']['Trial'] == trial_number:
                trial['metadata']['F_stim'] = F_stim
                break  # No need to search further once we've found and updated the trial

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_response_latency(trials):
    for stimulus_type, stimulus_trials in trials.items():
        for trial in stimulus_trials:
            F_stim = trial['metadata'].get('F_stim', None)
            valid_event_times = trial.get('valid_event_times', [])
            
            if F_stim is not None and len(valid_event_times) > 0:
                closest_event = min(valid_event_times, key=lambda x: abs(x - F_stim))
                
                # Check if the closest event is closer than 2000 frames
                if abs(closest_event - F_stim) < 2000:
                    trial['response_latency'] = closest_event - F_stim
                else:
                    trial['response_latency'] = np.nan
            else:
                trial['response_latency'] = np.nan

# Run the function to populate the 'response_latency' in each trial
calculate_response_latency(trials)

global_min = -1500
global_max = 1500

# Create uniform bins using global min and max
bins = np.linspace(global_min, global_max, 50)  # Change 50 to the number of bins you want

# Plotting
fig, axs = plt.subplots(len(trials), 1, figsize=(10, 40), sharex=True)

for i, (stimulus_type, stimulus_trials) in enumerate(trials.items()):
    latencies = [trial['response_latency'] for trial in stimulus_trials if not np.isnan(trial['response_latency'])]
    axs[i].hist(latencies, bins=bins, color=custom_colors.get(stimulus_type, 'b'))
    axs[i].set_title(f"Response Latency for {custom_labels.get(stimulus_type, stimulus_type)}")
    axs[i].set_xlabel("Response Latency (frames)")
    axs[i].set_ylabel("Frequency")
    axs[i].set_xlim([global_min, global_max])  # Set the same x-axis limits
    axs[i].set_ylim([0, 40])

plt.tight_layout()
plt.show()

In [ ]:
global_min = 0
global_max = 400

# Create uniform bins using global min and max
bins = np.linspace(global_min, global_max, 20)  # Change 50 to the number of bins you want

# Plotting
fig, axs = plt.subplots(len(trials), 1, figsize=(10, 40), sharex=True)

for i, (stimulus_type, stimulus_trials) in enumerate(trials.items()):
    latencies = [trial['response_latency'] for trial in stimulus_trials if not np.isnan(trial['response_latency'])]
    axs[i].hist(latencies, bins=bins, color=custom_colors.get(stimulus_type, 'b'))
    axs[i].set_title(f"Response Latency for {custom_labels.get(stimulus_type, stimulus_type)}")
    axs[i].set_xlabel("Response Latency (frames)")
    axs[i].set_ylabel("Frequency")
    axs[i].set_xlim([global_min, global_max])  # Set the same x-axis limits
    axs[i].set_ylim([0, 40])

plt.tight_layout()
plt.show()

In [ ]:
# Initialize lists to collect latencies for each cluster
cluster_0_latencies = []
cluster_neg1_latencies = []

# Loop through the trials to collect data
for stimulus_type, stimulus_trials in trials.items():
    for trial in stimulus_trials:
        F_stim = trial['metadata'].get('F_stim', None)
        valid_event_times = trial.get('valid_event_times', [])
        event_clusters = trial.get('event_clusters', [])

        if F_stim is not None:
            for i, event_time in enumerate(valid_event_times):
                latency = event_time - F_stim
                
                if event_clusters[i] == 0:
                    cluster_0_latencies.append(latency)
                elif event_clusters[i] == -1:
                    cluster_neg1_latencies.append(latency)

# Determine global min and max latencies for plotting
global_min = -1500
global_max = 1500

# Create uniform bins using global min and max
bins = np.linspace(global_min, global_max, 20)  # Change 50 to the number of bins you want

# Plotting
fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharex=True)

# Plot for Cluster 0
axs[0].hist(cluster_0_latencies, bins=bins, color=custom_colors.get('Cluster 0', 'b'))
axs[0].set_title("Latencies for Cluster 0")
axs[0].set_xlabel("Latency (frames)")
axs[0].set_ylabel("Frequency")

# Plot for Cluster -1
axs[1].hist(cluster_neg1_latencies, bins=bins, color=custom_colors.get('Cluster -1', 'r'))
axs[1].set_title("Latencies for Cluster -1")
axs[1].set_xlabel("Latency (frames)")
axs[1].set_ylabel("Frequency")

plt.tight_layout()
plt.show()


In [ ]:
# Initialize lists to collect latencies for each cluster
cluster_0_latencies = []
cluster_neg1_latencies = []

# Loop through the trials to collect data
for stimulus_type, stimulus_trials in trials.items():
    for trial in stimulus_trials:
        F_stim = trial['metadata'].get('F_stim', None)
        valid_event_times = trial.get('valid_event_times', [])
        event_clusters = trial.get('event_clusters', [])

        if F_stim is not None:
            for i, event_time in enumerate(valid_event_times):
                latency = event_time - F_stim
                
                if event_clusters[i] == 0:
                    cluster_0_latencies.append(latency)
                elif event_clusters[i] == -1:
                    cluster_neg1_latencies.append(latency)

# Determine global min and max latencies for plotting
global_min = 0
global_max = 400

# Create uniform bins using global min and max
bins = np.linspace(global_min, global_max, 20)  # Change 50 to the number of bins you want

# Plotting
fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharex=True)

# Plot for Cluster 0
axs[0].hist(cluster_0_latencies, bins=bins, color=custom_colors.get('Cluster 0', 'b'))
axs[0].set_title("Latencies for Cluster 0")
axs[0].set_xlabel("Latency (frames)")
axs[0].set_ylabel("Frequency")

# Plot for Cluster -1
axs[1].hist(cluster_neg1_latencies, bins=bins, color=custom_colors.get('Cluster -1', 'r'))
axs[1].set_title("Latencies for Cluster -1")
axs[1].set_xlabel("Latency (frames)")
axs[1].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Initialize a list to collect F_stim values
all_F_stims = []

# Loop through the trials to collect F_stim values
for stimulus_type, stimulus_trials in trials.items():
    for trial in stimulus_trials:
        F_stim = trial['metadata'].get('F_stim', None)
        if F_stim is not None:
            all_F_stims.append(F_stim)

# Plotting
plt.figure(figsize=(10, 6))

# Create the histogram
plt.hist(all_F_stims, bins=15, color='b')  # Change 50 to the number of bins you want

# Add title and labels
plt.title("Histogram of F_stim Values")
plt.xlabel("F_stim (frames)")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()
